# Notebook for CME Futures Challenge

# Downloading historical data for indices (S&P, NASDAQ, DJIA)

Imports

In [153]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import re
from typing import List, Dict

Make get_data function for downloading from yf

In [154]:
timeframe = '1000mo' # set timeframe

def get_data(tickers: List):
    data_dictionary = {}
    for ticker in tickers:
        data_dictionary[ticker] = yf.download(ticker, period=timeframe, interval='1d')
    return data_dictionary

# Getting Data

We need continuized data for returns information about futures since they roll over at expiration dates. I bought some continuous data for cheap, it's not back adjusted, which we could do - but I may try doing CV folds on each contract instead.

In [155]:
contracts = [
    # Equities
    'ES', # E S&P
    'NQ', # E Nasdaq
    'YM', # DJIA

    # Interest Rates
    'FF', # 30 days fed funds
    'TU', # 2yr us t note
    'FV', # 5yr us treasury note contracts
    'TY', # 10yr US treasury bond contracts
    'US', # 30yr US treasury bond contracts

    # Metals
    'GC', # Gold contracts
    'SI', # Silver
    'HG', # Copper
    'PL', # Platinum

    # Energy
    'CL', # Crude oil contracts
    'NG', # Natural Gas

    # Forex
    'JY', # JPY/USD contracts
    'BP', # BP/USD contracts
    'EU', # EURO/USD contracts
    'CD', # CAD/USD

    # Agriculture
    'C', # Corn
    'W', # Wheat
    'S', # Soybeans
    #'O', # Oats
    #'RR', # Rough Rice
    'LE', # Live Cattle
    'HE', # Lean Hogs
]

In [156]:
contract_data = {}

# Load in data
for contract in contracts:
    contract_data[contract] = pd.read_csv(f'futures_data/{contract}.txt')

Clean these up a bit

In [157]:
def rename_price_columns(data, item):
    # Rename columns
    data.rename(columns={'Open': f'{item}_open', 'High': f'{item}_high', 'Low': f'{item}_low', 'Close': f'{item}_close', 'Volume': f'{item}_volume'}, inplace=True)

In [158]:
for contract, data in contract_data.items():
    # Make sure date is datetime and set as index
    try:
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index(data['Date'], inplace=True)
        data.drop(columns=['Date'], inplace=True)
    except:
        print('Index already fixed.')

    # Rename columns
    rename_price_columns(data, contract)


In [159]:
contract_data['ES']

,ES_open,ES_high,ES_low,ES_close,ES_volume
Date,,,,,
1997-09-12,918.00,936.50,913.50,933.85,9945
1997-09-15,933.50,939.75,929.00,931.50,8911
1997-09-16,931.25,960.00,930.25,956.00,9945
1997-09-17,956.00,962.50,951.50,955.90,11093
1997-09-18,955.75,970.50,954.75,957.85,11620
...,...,...,...,...,...
2025-09-19,6700.00,6731.50,6679.50,6722.50,1230434
2025-09-22,6716.00,6756.75,6695.25,6752.50,1081569
2025-09-23,6749.50,6756.50,6701.75,6715.00,1264195


References for later

In [160]:
indexes = ['S&P', 'NASDAQ', 'DJIA']

Get yahoo finance data

In [161]:
tickers = ['^GSPC', '^IXIC', '^DJI'] # S&P, NASDAQ, DJIA

# Download data
data_dictionary = get_data(tickers)

# Turn to df
s_p = pd.DataFrame(data_dictionary['^GSPC'])
nasdaq = pd.DataFrame(data_dictionary['^IXIC'])
djia = pd.DataFrame(data_dictionary['^DJI'])

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3981970906.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_dictionary[ticker] = yf.download(ticker, period=timeframe, interval='1d')
[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3981970906.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_dictionary[ticker] = yf.download(ticker, period=timeframe, interval='1d')
[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3981970906.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_dictionary[ticker] = yf.download(ticker, period=timeframe, interval='1d')
[*********************100%***********************]  1 of 1 completed


Fix structure

In [162]:
# Flatten df so ticker doesn't span the top
s_p = s_p.droplevel(1, axis=1)
nasdaq = nasdaq.droplevel(1, axis=1)
djia = djia.droplevel(1, axis=1)

# Drop unnecessary columns
#s_p.drop(columns=['High', 'Low', 'Open'], inplace=True)
#nasdaq.drop(columns=['High', 'Low', 'Open'], inplace=True)
#djia.drop(columns=['High', 'Low', 'Open'], inplace=True)

index_data = {
    'S&P': s_p,
    'NASDAQ': nasdaq,
    'DJIA': djia,
}

for index, data in index_data.items():
    # Rename columns
    rename_price_columns(data, index)

# Downloading historical data for our factor model

We are going to model the index as a geometric brownian motion, with the mu factor being a linear regression model with numerous inputs.  

## Factor considerations:  
### <u>Term structure</u>
###### Term spread (10Y-3M)

### <u>Credit conditions</u>
###### IG spread (BAA-AAA)

### <u>Valuation</u>
###### Forward E/P - real 10Y
###### Dividend yield

### <u>Economic</u>
###### Fed funds
###### Inflation (CPI)
###### DXY change (dollar index)  

### Some of these we can get from yahoo finance:  

In [163]:
tickers = [
    # Term structure
    '^TNX', # 10yr CBOE
    '^IRX', # 3m bill (on discount basis, need to convert to yield)

    # Economic
    'DX-Y.NYB', # Dollar index

    # Volatility
    '^VIX',

    # Gold could be a measure of macro activity?
    "GLD", # Gold ETF

    # Other metals
    "XME", # SPDR Metals and mining ETF

    # Credit
    'FLRN', # Bloomberg IG Credit ETF
    'JNK', # Bloomberg HY Credit ETF

]

data_dictionary = get_data(tickers)

for ticker, data in data_dictionary.items():
    data_dictionary[ticker] = pd.DataFrame(data_dictionary[ticker])
    data_dictionary[ticker] = data_dictionary[ticker].droplevel(1, axis=1)
    rename_price_columns(data_dictionary[ticker], ticker)

ten_yr = data_dictionary['^TNX']
three_m = data_dictionary['^IRX']
dollar_index = data_dictionary['DX-Y.NYB']
vix = data_dictionary['^VIX']

# Need to drop volume column here
ten_yr.drop(columns=['^TNX_volume'], inplace=True)
three_m.drop(columns=['^IRX_volume'], inplace=True)
dollar_index.drop(columns=['DX-Y.NYB_volume'], inplace=True)
vix.drop(columns=['^VIX_volume'], inplace=True)


#gold = data_dictionary['GLD']
#metals = data_dictionary['XME']
#igb = pd.DataFrame(data_dictionary['FLRN'])
#hyb = data_dictionary['JNK']

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3981970906.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_dictionary[ticker] = yf.download(ticker, period=timeframe, interval='1d')
[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3981970906.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_dictionary[ticker] = yf.download(ticker, period=timeframe, interval='1d')
[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3981970906.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_dictionary[ticker] = yf.download(ticker, period=timeframe, interval='1d')
[*********************100%***********************]  1 of 1 completed
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3981970906.py:6: FutureWarning: YF.download() has changed argumen

In [164]:
yf_macros = [vix, ten_yr, three_m, dollar_index]

We should get dividend yield too

In [165]:
div_data = {}
etfs = ['SPY', 'QQQ', 'DIA']

for etf in etfs:
    ticker = yf.Ticker(etf)
    div = ticker.dividends
    price = ticker.history(timeframe)['Close']

    # Calculate dividend yield
    div_12m = div.rolling(window='365D', min_periods=1).sum()
    div_12m = div_12m.reindex(price.index, method='ffill')
    div_yield = div_12m / price
    div_data[etf] = div_yield

Fix index for all 3 and rename columns

In [166]:
div_data['SPY'].index = pd.to_datetime(div_data['SPY'].index).normalize().tz_localize(None) # Normalize puts date in format we want
div_data['QQQ'].index = pd.to_datetime(div_data['QQQ'].index).normalize().tz_localize(None) # Localize (none) makes sure it doesn't add our timezone
div_data['DIA'].index = pd.to_datetime(div_data['DIA'].index).normalize().tz_localize(None)

div_data['SPY'].name = 'S&P_div'
div_data['QQQ'].name = 'NASDAQ_div'
div_data['DIA'].name = 'DJIA_div'

### pandas_datareader lets us download fred data

In [167]:
from pandas_datareader import data as pdr
from datetime import datetime

In [168]:
start = datetime(1990,1,1) # Start date for download

# Macroeconomic data
gdp = pdr.DataReader("GDP", "fred", start)
cpi = pdr.DataReader("CPIAUCSL", "fred", start)
fedfunds = pdr.DataReader("FEDFUNDS", "fred", start)
consumer_sentiment = pdr.DataReader("UMCSENT", "fred", start)
inflation_expectation = pdr.DataReader("MICH", "fred", start)

# For some reason this download doesn't have the most recent fed funds rate
fedfunds = pd.concat([fedfunds['FEDFUNDS'], pd.Series([4.08], index=[datetime(2025,9,17)])])

# Credit risk data
ig_spread = pdr.DataReader("BAMLC0A4CBBB", "fred", start)   # BofA BBB corp minus Treasuries
#hy_spread = pdr.DataReader("BAMLH0A0HYM2", "fred", start)   # BofA US High Yield spread
#baa_spread = pdr.DataReader("BAA10Y", "fred", start)        # Moody’s Baa – 10Y Treasury

In [169]:
fred_data = [gdp, cpi, fedfunds, ig_spread, consumer_sentiment, inflation_expectation]

In [170]:
# Last business day <= today
last_bday = pd.bdate_range(end=pd.Timestamp.today().normalize().tz_localize(None), periods=1)[0]

for i, df in enumerate(fred_data):
    s = df.squeeze() # make it a Series
    # Build a business-day index from the series start to last_bday
    bidx = pd.bdate_range(start=s.index.min(), end=last_bday)
    # Reindex to business days and forward-fill
    s = s.reindex(bidx, method='ffill')
    # Write back as a 1-col DataFrame with a proper name
    name = s.name if s.name else f"series_{i}"
    fred_data[i] = s.to_frame(name)

Let's build a master dataframe

In [171]:
pd.set_option('display.max_columns', None)
data = s_p.join([nasdaq, djia])
data = data.join(contract for contract in contract_data.values())
data = data.join(macro for macro in yf_macros)
data = data.join(div for div in div_data.values())
data = data.join([fd for fd in fred_data])

Rename some columns

In [172]:
data.rename(columns={'CPIAUCSL': 'CPI', 'BAMLC0A4CBBB': 'credit_spread', 'UMCSENT': 'consumer_sentiment', 'MICH': 'inflation_expectation', 'series_2': 'fed_funds'}, inplace=True)
data[data.index.year > 2000]

,S&P_close,S&P_high,S&P_low,S&P_open,S&P_volume,NASDAQ_close,NASDAQ_high,NASDAQ_low,NASDAQ_open,NASDAQ_volume,DJIA_close,DJIA_high,DJIA_low,DJIA_open,DJIA_volume,ES_open,ES_high,ES_low,ES_close,ES_volume,NQ_open,NQ_high,NQ_low,NQ_close,NQ_volume,YM_open,YM_high,YM_low,YM_close,YM_volume,FF_open,FF_high,FF_low,FF_close,FF_volume,TU_open,TU_high,TU_low,TU_close,TU_volume,FV_open,FV_high,FV_low,FV_close,FV_volume,TY_open,TY_high,TY_low,TY_close,TY_volume,US_open,US_high,US_low,US_close,US_volume,GC_open,GC_high,GC_low,GC_close,GC_volume,SI_open,SI_high,SI_low,SI_close,SI_volume,HG_open,HG_high,HG_low,HG_close,HG_volume,PL_open,PL_high,PL_low,PL_close,PL_volume,CL_open,CL_high,CL_low,CL_close,CL_volume,NG_open,NG_high,NG_low,NG_close,NG_volume,JY_open,JY_high,JY_low,JY_close,JY_volume,BP_open,BP_high,BP_low,BP_close,BP_volume,EU_open,EU_high,EU_low,EU_close,EU_volume,CD_open,CD_high,CD_low,CD_close,CD_volume,C_open,C_high,C_low,C_close,C_volume,W_open,W_high,W_low,W_close,W_volume,S_open,S_high,S_low,S_close,S_volume,LE_open,LE_high,LE_low,LE_close,LE_volume,HE_open,HE_high,HE_low,HE_close,HE_volume,^VIX_close,^VIX_high,^VIX_low,^VIX_open,^TNX_close,^TNX_high,^TNX_low,^TNX_open,^IRX_close,^IRX_high,^IRX_low,^IRX_open,DX-Y.NYB_close,DX-Y.NYB_high,DX-Y.NYB_low,DX-Y.NYB_open,S&P_div,NASDAQ_div,DJIA_div,GDP,CPI,fed_funds,credit_spread,consumer_sentiment,inflation_expectation
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001-01-02,1283.270020,1320.280029,1276.050049,1320.280029,1.129400e+09,2291.860107,2474.159912,2273.070068,2474.159912,1.918930e+09,10646.150391,10797.019531,10585.360352,10790.919922,253300000.0,1331.00,1336.75,1289.25,1299.8,92668.0,2375.0,2414.0,2130.5,2167.5,87316.0,NaN,NaN,NaN,NaN,NaN,94.0850,94.0850,94.0850,94.0850,24.0,101.687500,102.070297,101.687500,102.062500,6759.0,103.781197,104.484398,103.687500,104.437500,87216.0,104.859398,106.062500,104.843803,105.984398,16102.0,104.5625,105.9062,104.5625,105.7812,18069.0,273.8,275.0,269.0,270.0,41621.0,4.640,4.650,4.590,4.595,6916.0,0.8400,0.8400,0.8100,0.8100,3519.0,608.1,611.0,604.5,608.9,0.0,26.90,27.40,26.65,27.21,52321.0,9.450,9.450,8.250,8.364,7870.0,0.008800,0.008800,0.008800,0.008800,0.0,1.4938,1.4978,1.4912,1.4932,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,231.50,232.00,226.75,227.25,42257.0,278.50,280.00,275.00,276.25,14282.0,510.00,510.50,503.50,504.50,22643.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.990000,30.400000,27.920000,27.930000,4.917,5.075,4.894,5.075,5.680,5.720,5.650,5.710,108.769997,109.650002,108.639999,109.330002,0.022532,NaN,0.025006,10470.231,175.600,5.98,2.70,94.7,3.0
2001-01-03,1347.560059,1347.760010,1274.619995,1283.270020,1.880700e+09,2616.689941,2618.030029,2251.709961,2254.560059,3.188000e+09,10945.750000,11019.049805,10581.089844,10637.419922,420720000.0,1301.00,1367.25,1288.75,1359.2,130684.0,2590.5,2590.0,2529.5,2531.5,115138.0,NaN,NaN,NaN,NaN,NaN,94.3450,94.3450,94.3450,94.3450,24.0,102.125000,102.218803,101.812500,101.882797,8265.0,104.500000,104.531197,103.343803,103.515602,139562.0,106.171898,106.234398,105.984398,106.140602,6234.0,106.2500,106.3125,106.0938,106.2812,3407.0,270.0,271.0,267.6,269.3,37296.0,4.595,4.605,4.535,4.540,8952.0,0.8100,0.8200,0.8000,0.8200,14628.0,610.0,624.0,608.0,619.4,0.0,27.21,28.14,27.08,28.00,66628.0,8.300,8.700,8.110,8.189,13754.0,0.008800,0.008900,0.008800,0.008900,0.0,1.5026,1.5110,1.4990,1.5080,0.0,NaN,NaN,NaN,NaN,NaN,0.67090,0.67190,0.66910,0.66930,0.0,227.00,230.75,226.50,230.00,39703.0,276.25,284.00,275.50,283.25,28519.0,504.00,508.00,501.25,506.75,32700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.600000,29.959999,25.990000,29.629999,5.097,5.134,4.874,4.882,5.530,5.760,5.530,5.680,110.169998,110.339996,108.089996,108.589996,0.021499,NaN,0.024293,10470.231,175.600,5.98,2.67,94.7,3.0
2001-01-04,1333.339966,1350.239990,1329.140015,1347.560059,2.131000e+09,2566.830078,2644.800049

# Linear regression model

### Preprocessing Data

Let's check for NaNs (there's a ton)

In [173]:
data.isna().sum()

S&P_close                    0
S&P_high                     0
S&P_low                      0
S&P_open                     0
S&P_volume                   0
                         ...  
CPI                      11947
fed_funds                11947
credit_spread            13719
consumer_sentiment       11947
inflation_expectation    11947
Length: 155, dtype: int64

Check data types

In [174]:
data.dtypes.unique()

array([dtype('float64')], dtype=object)

In [175]:
data = data[data.columns].astype(float)

Fill any gaps

In [176]:
data = data.replace(0, np.nan)
data = data.fillna(method='ffill')

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\1584352194.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


Drop others

In [177]:
data = data.dropna().copy()
data

,S&P_close,S&P_high,S&P_low,S&P_open,S&P_volume,NASDAQ_close,NASDAQ_high,NASDAQ_low,NASDAQ_open,NASDAQ_volume,DJIA_close,DJIA_high,DJIA_low,DJIA_open,DJIA_volume,ES_open,ES_high,ES_low,ES_close,ES_volume,NQ_open,NQ_high,NQ_low,NQ_close,NQ_volume,YM_open,YM_high,YM_low,YM_close,YM_volume,FF_open,FF_high,FF_low,FF_close,FF_volume,TU_open,TU_high,TU_low,TU_close,TU_volume,FV_open,FV_high,FV_low,FV_close,FV_volume,TY_open,TY_high,TY_low,TY_close,TY_volume,US_open,US_high,US_low,US_close,US_volume,GC_open,GC_high,GC_low,GC_close,GC_volume,SI_open,SI_high,SI_low,SI_close,SI_volume,HG_open,HG_high,HG_low,HG_close,HG_volume,PL_open,PL_high,PL_low,PL_close,PL_volume,CL_open,CL_high,CL_low,CL_close,CL_volume,NG_open,NG_high,NG_low,NG_close,NG_volume,JY_open,JY_high,JY_low,JY_close,JY_volume,BP_open,BP_high,BP_low,BP_close,BP_volume,EU_open,EU_high,EU_low,EU_close,EU_volume,CD_open,CD_high,CD_low,CD_close,CD_volume,C_open,C_high,C_low,C_close,C_volume,W_open,W_high,W_low,W_close,W_volume,S_open,S_high,S_low,S_close,S_volume,LE_open,LE_high,LE_low,LE_close,LE_volume,HE_open,HE_high,HE_low,HE_close,HE_volume,^VIX_close,^VIX_high,^VIX_low,^VIX_open,^TNX_close,^TNX_high,^TNX_low,^TNX_open,^IRX_close,^IRX_high,^IRX_low,^IRX_open,DX-Y.NYB_close,DX-Y.NYB_high,DX-Y.NYB_low,DX-Y.NYB_open,S&P_div,NASDAQ_div,DJIA_div,GDP,CPI,fed_funds,credit_spread,consumer_sentiment,inflation_expectation
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003-12-24,1094.040039,1096.400024,1092.729980,1096.020020,5.180600e+08,1969.229980,1974.310059,1964.880005,1969.709961,6.426300e+08,10305.190430,10341.480469,10305.120117,10341.410156,86180000.0,1093.50,1095.25,1089.50,1093.00,94667.0,1444.5,1451.0,1441.5,1446.50,47758.0,10290.0,10309.0,10272.0,10281.0,10710.0,98.9800,98.9800,98.9800,98.9800,90.0,107.187500,107.187500,107.078102,107.101601,1203.0,111.312500,111.781303,111.234398,111.718803,51220.0,112.031197,112.765602,111.937500,112.625000,97080.0,109.62500,110.53120,109.5312,110.28120,34715.0,411.5,413.5,411.3,412.8,18163.0,5.750,5.805,5.735,5.793,6923.0,1.0200,1.0200,1.0100,1.0200,6363.0,808.0,825.0,808.0,821.5,1407.0,32.06,33.00,31.88,32.86,60080.0,6.260,6.440,6.110,6.392,18341.0,0.009300,0.009400,0.009300,0.009300,4096.0,1.7534,1.7654,1.7526,1.7644,3450.0,1.2373,1.2444,1.23690,1.2426,13073.0,0.75370,0.76280,0.75340,0.76230,5352.0,230.00,238.50,229.00,235.75,97419.0,359.0,369.00,358.00,363.00,23792.0,733.00,767.00,733.00,765.00,34591.0,90.850,90.85,90.850,90.850,1.0,51.600,52.000,51.075,51.200,67.0,16.660000,16.770000,16.240000,16.650000,4.187,4.246,4.182,4.240,0.872,0.877,0.872,0.875,87.540001,87.970001,87.449997,87.949997,0.028232,0.000461,0.033565,11772.234,185.500,0.98,1.33,92.6,2.6
2003-12-26,1095.890015,1098.469971,1094.040039,1094.040039,3.560700e+08,1973.140015,1979.739990,1970.369995,1970.369995,5.308100e+08,10324.669922,10343.040039,10304.820312,10305.849609,49520000.0,1093.50,1097.00,1092.50,1093.50,68120.0,1449.0,1454.5,1444.0,1445.00,31574.0,10289.0,10318.0,10285.0,10292.0,6721.0,98.9850,98.9850,98.9850,98.9850,3.0,107.187500,107.187500,107.078102,107.101601,1203.0,111.734398,111.984398,111.671898,111.968803,16885.0,112.734299,113.062500,112.578102,113.000000,44915.0,110.50000,110.93750,110.2812,110.90620,18523.0,411.5,413.5,411.3,412.8,18163.0,5.750,5.805,5.735,5.793,6923.0,1.0200,1.0200,1.0100,1.0200,6363.0,808.0,825.0,808.0,821.5,1407.0,32.06,33.00,31.88,32.86,60080.0,6.260,6.440,6.110,6.392,18341.0,0.009400,0.009400,0.009400,0.009400,2353.0,1.7630,1.7655,1.7601,1.7610,665.0,1.2422,1.2436,1.24000,1.2408,13073.0,0.76070,0.76440,0.76030,0.76220,2293.0,234.00,240.50,232.75,239.75,62585.0,362.5,369.50,362.00,366.00,11590.0,768.00,801.00,766.00,800.00,38149.0,87.850,87.85,87.850,87.850,1.0,51.900,53.200,51.900,53.200,39.0,17.450001,17.490000,16.629999,16.760000,4.148,4.176,4.147,4.176,0.847,0.866,0.833,0.833,87.629997,87.650002,87.430000,87.

# Feature Engineering

We need to be careful to not include things such as raw moving averages that will leak volatility information into our drift prediction  

Forward adjusting futures

In [178]:
# Kibot continuous futures rollover rules
# months_off = 0  -> X trading days BEFORE contract expiration
# months_off = 1  -> X trading days FROM the LAST BUSINESS DAY of the PRIOR month
# months_off = 0 & days_off = 0 -> roll ON contract expiration

roll_rules = {
    # ——— Equities ———
    'ES': (0, 5),  # 5 trading days before expiration
    'NQ': (0, 5),  # 5 trading days before expiration
    'YM': (0, 5),  # 5 trading days before expiration

    # ——— Interest Rates ———
    'FF': (0, 0),  # on contract expiration
    'TU': (1, 1),  # 1 trading day from prior month-end
    'FV': (1, 1),  # 1 trading day from prior month-end
    'TY': (1, 1),  # 1 trading day from prior month-end
    'US': (1, 1),  # 1 trading day from prior month-end

    # ——— Metals ———
    'GC': (1, 2),  # 2 trading days from prior month-end
    'SI': (1, 2),  # 2 trading days from prior month-end
    'HG': (1, 2),  # 2 trading days from prior month-end
    'PL': (1, 2),  # 2 trading days from prior month-end

    # ——— Energy ———
    'CL': (0, 3),  # 3 trading days before expiration
    'NG': (0, 3),  # 3 trading days before expiration

    # ——— Forex ———
    'JY': (0, 2),  # 2 trading days before expiration
    'BP': (0, 2),  # 2 trading days before expiration
    'EU': (0, 2),  # 2 trading days before expiration
    'CD': (0, 3),  # 3 trading days before expiration

    # ——— Agriculture ———
    'C' : (1, 8),  # 8 trading days from prior month-end
    'W' : (1, 8),  # 8 trading days from prior month-end
    'S' : (1, 8),  # 8 trading days from prior month-end
    'O' : (1, 8),  # 8 trading days from prior month-end
    'RR': (1, 8),  # 8 trading days from prior month-end
    'LE': (1,10),  # 10 trading days from prior month-end
    'HE': (1, 6),  # 6 trading days from prior month-end
}


In [179]:
# Useful sets
IMM_QTR = {3, 6, 9, 12}
ALL_MONTHS = set(range(1, 13))

DELIVERY_MONTHS = {
    # ——— Equities (CME) ———
    'ES': IMM_QTR,                          # Mar, Jun, Sep, Dec
    'NQ': IMM_QTR,                          # Mar, Jun, Sep, Dec
    'YM': IMM_QTR,                          # Mar, Jun, Sep, Dec

    # ——— Interest Rates (CBOT/CME) ———
    'FF': ALL_MONTHS,                       # Monthly
    'TU': IMM_QTR,                          # 2Y Note: Mar, Jun, Sep, Dec
    'FV': IMM_QTR,                          # 5Y Note: Mar, Jun, Sep, Dec
    'TY': IMM_QTR,                          # 10Y Note: Mar, Jun, Sep, Dec
    'US': IMM_QTR,                          # 30Y Bond: Mar, Jun, Sep, Dec

    # ——— Metals (COMEX/NYMEX) ———
    'GC': {2, 4, 6, 8, 10, 12},             # Gold: Feb, Apr, Jun, Aug, Oct, Dec
    'SI': {1, 3, 5, 7, 9, 12},              # Silver: Jan, Mar, May, Jul, Sep, Dec
    'HG': {3, 5, 7, 9, 12},                 # Copper: Mar, May, Jul, Sep, Dec
    'PL': {1, 4, 7, 10},                    # Platinum: Jan, Apr, Jul, Oct

    # ——— Energy (NYMEX) ———
    'CL': ALL_MONTHS,                       # Crude Oil: monthly
    'NG': ALL_MONTHS,                       # Nat Gas: monthly

    # ——— Forex (CME) ———
    'JY': IMM_QTR,                          # JPY/USD: Mar, Jun, Sep, Dec
    'BP': IMM_QTR,                          # GBP/USD: Mar, Jun, Sep, Dec
    'EU': IMM_QTR,                          # EUR/USD: Mar, Jun, Sep, Dec
    'CD': IMM_QTR,                          # CAD/USD: Mar, Jun, Sep, Dec

    # ——— Grains & Oilseeds (CBOT) ———
    'C' : {3, 5, 7, 9, 12},                 # Corn: Mar, May, Jul, Sep, Dec
    'W' : {3, 5, 7, 9, 12},                 # Wheat: Mar, May, Jul, Sep, Dec
    'S' : {1, 3, 5, 7, 8, 9, 11},           # Soybeans: Jan, Mar, May, Jul, Aug, Sep, Nov
    'O' : {3, 5, 7, 9, 12},                 # Oats: Mar, May, Jul, Sep, Dec
    'RR': {1, 3, 5, 7, 9, 11},              # Rough Rice: Jan, Mar, May, Jul, Sep, Nov

    # ——— Livestock (CME) ———
    'LE': {2, 4, 6, 8, 10, 12},             # Live Cattle: Feb, Apr, Jun, Aug, Oct, Dec
    'HE': {2, 4, 5, 6, 7, 8, 10, 12},       # Lean Hogs: Feb, Apr, May, Jun, Jul, Aug, Oct, Dec
}


In [ ]:
# -----------------------
# Quarterly anchors
# -----------------------
def _third_fridays_quarterly(start, end):
    d = pd.date_range(start, end, freq="WOM-3FRI")
    return d[d.month.isin(IMM_QTR)]

def _bmonth_end_quarterly(start, end):
    all_month_ends = pd.date_range(start.normalize(), end.normalize(), freq="BM")
    return all_month_ends[all_month_ends.month.isin(IMM_QTR)]

def _third_wednesdays_quarterly(start, end):
    weds = pd.date_range(start, end, freq="WOM-3WED")
    return weds[weds.month.isin(IMM_QTR)]

# -----------------------
# Monthly helpers
# -----------------------
def _bmonth_end_monthly(start, end):
    return pd.date_range(start.normalize(), end.normalize(), freq="BM")

def _bm_end_selected_months(start, end, months_set):
    bms = pd.date_range(start.normalize(), end.normalize(), freq="BM")
    return bms[bms.month.isin(months_set)]

# -----------------------
# Product-specific expiries
# -----------------------
def _gold_expiries(start, end):
    # 3rd-last BUSINESS day in GC delivery months (even months)
    mends = pd.date_range(start.normalize(), end.normalize(), freq="BM")
    mends = mends[mends.month.isin(DELIVERY_MONTHS['GC'])]
    return (mends - pd.offsets.BusinessDay(2))

def _cl_expiries(start, end):
    # Approx: 3 business days before the 25th of the PRIOR month for each delivery month
    cl_expiries = []
    month_starts = pd.date_range(start.normalize(), end.normalize(), freq="MS")
    for ms in month_starts:
        prior_month_25th = (ms - pd.offsets.MonthBegin(1)) + pd.offsets.Day(24)
        prev_bd = prior_month_25th
        while prev_bd.weekday() >= 5:
            prev_bd -= pd.Timedelta(days=1)
        ltd = prev_bd - pd.offsets.BusinessDay(3)
        cl_expiries.append(ltd.normalize())
    return pd.DatetimeIndex(sorted(set(cl_expiries)))

def _ng_expiries(start, end):
    # Approx: 3 business days BEFORE the 1st calendar day of the delivery month (MS - 3 BDays)
    ng_exp = []
    month_starts = pd.date_range(start.normalize(), end.normalize(), freq="MS")
    for ms in month_starts:
        ltd = ms - pd.offsets.BusinessDay(3)
        ng_exp.append(ltd.normalize())
    return pd.DatetimeIndex(sorted(set(ng_exp)))

# -----------------------
# Master expiry calendar router (now respects DELIVERY_MONTHS)
# -----------------------
def _expiry_calendar(symbol, start, end):
    s = symbol.upper()

    # Equities (quarterly IMM)
    if s in {"ES", "NQ", "YM"}:
        return _third_fridays_quarterly(start, end)

    # Interest Rates
    if s in {"TU", "FV", "TY", "US"}:
        return _bmonth_end_quarterly(start, end)
    if s == "FF":
        return _bmonth_end_monthly(start, end)

    # Metals
    if s == "GC":
        return _gold_expiries(start, end)
    if s in {"SI", "HG", "PL"}:
        # anchor on last BD of each listed delivery month
        return _bm_end_selected_months(start, end, DELIVERY_MONTHS[s])

    # Energy
    if s == "CL":
        return _cl_expiries(start, end)
    if s == "NG":
        return _ng_expiries(start, end)

    # FX (quarterly IMM, 3rd Wednesday anchor is fine for IMM month identification)
    if s in {"JY", "BP", "EU", "CD"}:
        return _third_wednesdays_quarterly(start, end)

    # Agriculture & Livestock: last BD of listed delivery months only
    if s in DELIVERY_MONTHS:
        return _bm_end_selected_months(start, end, DELIVERY_MONTHS[s])

    # Fallback: quarterly 3rd Fridays
    return _third_fridays_quarterly(start, end)

# -----------------------
# main: compute rollover flags, episodes, and DTR
# -----------------------
def get_rollover_data(data: pd.DataFrame, item: str, roll_rules: dict):
    months_off, days_off = roll_rules[item]  # (months, days)

    # buffer to see next expiry beyond dataset end
    start = pd.Timestamp(data.index.min()).normalize()
    end = (pd.Timestamp(data.index.max()).normalize() + pd.DateOffset(months=4))

    expiries = _expiry_calendar(item, start, end)

    # Compute roll dates per rule
    if months_off == 0:
        roll_dates = expiries - pd.offsets.BusinessDay(days_off)
    else:
        # Go back N months to that month's LAST business day, then step back 'days_off'
        last_bds = []
        for e in expiries:
            mstart = (e - pd.offsets.MonthBegin(months_off))
            lbd = (mstart + pd.offsets.BMonthEnd(0))
            last_bds.append(lbd)
        anchor_last_bd = pd.DatetimeIndex(last_bds)
        roll_dates = anchor_last_bd - pd.offsets.BusinessDay(days_off)

    roll_dates = pd.DatetimeIndex(sorted(set(pd.to_datetime(roll_dates))))
    roll_dates = roll_dates.intersection(pd.DatetimeIndex(data.index))

    # Flags and episodes
    rflag = f"{item}_Rollover"
    eph = f"{item}_Episode"
    data[rflag] = 0
    if len(roll_dates):
        data.loc[roll_dates, rflag] = 1
    data[eph] = data[rflag].cumsum()

    # Days until next rollover (trading days)
    next_roll = (
        pd.Series(roll_dates, index=roll_dates)
        .reindex(data.index, method="bfill")
        .to_numpy()
    )
    days = np.full(len(data), np.nan)
    mask = ~pd.isna(next_roll)
    left = data.index.to_numpy(dtype="datetime64[D]")
    right = pd.to_datetime(next_roll).to_numpy(dtype="datetime64[D]")
    days[mask] = np.busday_count(left[mask], right[mask])  # excludes right endpoint
    data[f"{item}_Days_Until_Rollover"] = days

# -----------------------
# forward adjustment on roll days
# -----------------------
def rollover_forward_adjustment(data: pd.DataFrame, contract: str):
    rflag = f"{contract}_Rollover"
    close = f"{contract}_close"
    jump = data[rflag].fillna(0).astype(int) * (data[close] - data[close].shift(1).fillna(0.0))
    cum_adj = jump.cumsum()
    for col in ["open", "high", "low", "close"]:
        c = f"{contract}_{col}"
        if c in data.columns:
            data[c] = data[c] - cum_adj
    cols = [f'{contract}_{col}' for col in ["open", "high", "low", "close"]]
    data[cols] = data[cols] + abs(data[cols].min().min())


In [184]:
for contract in contracts:
    # Set info about expirations
    get_rollover_data(data, contract, roll_rules)

    # Forward adjust rollover
    rollover_forward_adjustment(data, contract)

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3994407707.py:20: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  return pd.date_range(start.normalize(), end.normalize(), freq="BM")
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3994407707.py:9: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  all_month_ends = pd.date_range(start.normalize(), end.normalize(), freq="BM")
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3994407707.py:9: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  all_month_ends = pd.date_range(start.normalize(), end.normalize(), freq="BM")
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3994407707.py:9: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  all_month_ends = pd.date_range(start.normalize(), end.normalize(), freq="BM")
C:\Users\rjsyo\Ap

In [190]:
data['NG_close']

Date
2003-12-24    11.501
2003-12-26    11.501
2003-12-29    11.352
2003-12-30    11.709
2003-12-31    11.298
               ...  
2025-09-29     0.634
2025-09-30     0.670
2025-10-01     0.843
2025-10-02     0.843
2025-10-03     0.843
Name: NG_close, Length: 5479, dtype: float64

Function definitions to help out

In [123]:
def rolling_mean(data, window):
    return data.rolling(window, min_periods=window).mean()

### Features

Technical Indicators

In [124]:
def add_technical_indicators(data, items, cfg=None):
    if cfg is None:
        cfg = {
            "rsi_n": 14,
            "macd_fast": 12, "macd_slow": 26, "macd_signal": 9,
            "bb_n": 20, "bb_k": 2.0,
            "atr_n": 14
        }
    out = data.copy()

    for item in items:
        o = out[f"{item}_open"].astype(float)
        h = out[f"{item}_high"].astype(float)
        l = out[f"{item}_low"].astype(float)
        c = out[f"{item}_close"].astype(float)
        v = out[f"{item}_volume"].astype(float)

        # --- 1) RSI (Wilder smoothing) ---
        n = cfg["rsi_n"]
        delta = c.diff()
        gain = delta.clip(lower=0.0)
        loss = (-delta).clip(lower=0.0)
        avg_gain = gain.ewm(alpha=1/n, adjust=False, min_periods=n).mean()
        avg_loss = loss.ewm(alpha=1/n, adjust=False, min_periods=n).mean()
        rs = avg_gain / (avg_loss.replace(0, np.nan))
        rsi = 100 - (100 / (1 + rs))
        out[f"{item}_rsi_{n}"] = rsi

        # --- 2) MACD (EMA fast/slow, signal, hist) ---
        ema_fast = c.ewm(span=cfg["macd_fast"], adjust=False).mean()
        ema_slow = c.ewm(span=cfg["macd_slow"], adjust=False).mean()
        macd = ema_fast - ema_slow
        macd_signal = macd.ewm(span=cfg["macd_signal"], adjust=False).mean()
        macd_hist = macd - macd_signal
        out[f"{item}_macd"] = macd
        out[f"{item}_macd_signal"] = macd_signal
        out[f"{item}_macd_hist"] = macd_hist

        # --- 3) Bollinger Bands (+ %B, Bandwidth) ---
        bb_n = cfg["bb_n"]; bb_k = cfg["bb_k"]
        mavg = c.rolling(bb_n, min_periods=bb_n).mean()
        mstd = c.rolling(bb_n, min_periods=bb_n).std(ddof=0)
        upper = mavg + bb_k * mstd
        lower = mavg - bb_k * mstd
        pct_b = (c - lower) / (upper - lower)
        bandwidth = (upper - lower) / mavg
        out[f"{item}_bb_mean_{bb_n}"] = mavg
        out[f"{item}_bb_upper_{bb_n}_{int(bb_k)}"] = upper
        out[f"{item}_bb_lower_{bb_n}_{int(bb_k)}"] = lower
        out[f"{item}_bb_percent_b"] = pct_b
        out[f"{item}_bb_bandwidth"] = bandwidth

        # --- 4) ATR (Wilder, uses True Range) ---
        prev_c = c.shift(1)
        tr = np.maximum(h - l, np.maximum((h - prev_c).abs(), (l - prev_c).abs()))
        atr = tr.ewm(alpha=1/cfg["atr_n"], adjust=False, min_periods=cfg["atr_n"]).mean()
        out[f"{item}_atr_{cfg['atr_n']}"] = atr

        # --- 5) OBV (On-Balance Volume) ---
        sign = np.sign(c.diff().fillna(0.0))
        obv = (sign * v).fillna(0.0).cumsum()
        out[f"{item}_obv"] = obv

    return out

In [125]:
data = add_technical_indicators(data, contracts)

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\1527905683.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{item}_obv"] = obv
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\1527905683.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[f"{item}_rsi_{n}"] = rsi
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\1527905683.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once 

In [126]:
def get_log_price(data, items):
    data = data.copy()
    
    ohlcv = ['open', 'high', 'low', 'close', 'volume']
    for item in items:
        data[[f'{item}_log_{key}' for key in ohlcv]] = np.log(data[[f'{item}_{key}' for key in ohlcv]])
        data.drop(columns=[f'{item}_{key}' for key in ohlcv], inplace=True) # Drop originals
    return data

In [127]:
def get_log_returns(data, items):
    data = data.copy()
    
    ohlcv = ['open', 'high', 'low', 'close', 'volume']
    for item in items:
        for key in ohlcv:
            data[f'{item}_log_{key}_ret'] = data[f'{item}_log_{key}'].diff()

        data[f'{item}_log_open_close_ret'] = data[f'{item}_log_close'] - data[f'{item}_log_open']
    return data

In [128]:
# Get log transforms of price data
log_price_keys = indexes + list(contracts) #+ ['GLD', 'XME', 'JNK']

# Log prices
data = get_log_price(data, log_price_keys)

# Log returns
data = get_log_returns(data, log_price_keys)

# Log for other price data (GDP)
data['log_GDP'] = np.log(data['GDP'])
data.drop(columns=['GDP'], inplace=True)

data = data.replace(0, np.nan)
data = data.fillna(method='ffill').fillna(0).copy()

C:\Users\rjsyo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\internals\blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\823185119.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[[f'{item}_log_{key}' for key in ohlcv]] = np.log(data[[f'{item}_{key}' for key in ohlcv]])
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\823185119.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-

Price/Volume Features

In [129]:
def get_price_volume_features(data, items):
    data = data.copy()
    
    for item in items:
        # Price-based
        data[f'{item}_mom_1w'] = data[f'{item}_log_close'].diff(5) # Total price change / momentum indicator
        data[f'{item}_mom_3m'] = data[f'{item}_log_close'].diff(63)
        data[f'{item}_1m_rolling_price'] = rolling_mean(data[f'{item}_log_close'], 21)
        data[f'{item}_3m_rolling_price'] = rolling_mean(data[f'{item}_log_close'], 63)
        data[f'{item}_6m_rolling_price'] = rolling_mean(data[f'{item}_log_close'], 126)
        data[f'{item}_trend_speed_price'] = data[f'{item}_3m_rolling_price'].diff(5)  # How fast the 3m trend is changing on a weekly basis
        data[f'{item}_trend_dist_price'] = data[f'{item}_log_close'] - data[f'{item}_3m_rolling_price']

        # Volume-based (essentially the same as price for now)
        data[f'{item}_vlm_1w'] = data[f'{item}_log_volume'].diff(5) # Total volume change / momentum indicator
        data[f'{item}_vlm_1m'] = data[f'{item}_log_volume'].diff(21)
        data[f'{item}_vlm_3m'] = data[f'{item}_log_volume'].diff(63)
        data[f'{item}_3m_rolling_volume'] = rolling_mean(data[f'{item}_log_volume'], 63)
        data[f'{item}_trend_speed_volume'] = data[f'{item}_3m_rolling_volume'].diff(5)  # How fast the 3m trend is changing on a weekly basis
        data[f'{item}_trend_dist_volume'] = data[f'{item}_log_volume'] - data[f'{item}_3m_rolling_volume']

    return data

In [130]:
data = get_price_volume_features(data, log_price_keys)

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\550107399.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{item}_vlm_1w'] = data[f'{item}_log_volume'].diff(5) # Total volume change / momentum indicator
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\550107399.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{item}_vlm_1m'] = data[f'{item}_log_volume'].diff(21)
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\550107399.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

Macro features

In [131]:
data['log_GDP_ret'] = data['log_GDP'].diff()
data['log_GDP_ret_1y'] = data['log_GDP'].diff(252)
data['inflation_expectation_ret'] = data['inflation_expectation'].pct_change()
data['consumer_sentiment_ret'] = data['consumer_sentiment'].pct_change()
data['cpi_ret'] = data['CPI'].pct_change()

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\688002531.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['log_GDP_ret'] = data['log_GDP'].diff()
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\688002531.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['log_GDP_ret_1y'] = data['log_GDP'].diff(252)
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\688002531.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  C

Spreads

In [132]:
import itertools

In [133]:
def generate_spreads(data: pd.DataFrame, contracts=contracts, ret_suffix="_log_close_ret", vol_suffix="_3m_rolling_volume"):
    for c1, c2 in itertools.combinations(contracts, 2):
        ret_col1, ret_col2 = f"{c1}{ret_suffix}", f"{c2}{ret_suffix}"
        vol_col1, vol_col2 = f"{c1}{vol_suffix}", f"{c2}{vol_suffix}"

        if ret_col1 in data.columns and ret_col2 in data.columns:
            spread_ret_col = f"Spread_{c1}_{c2}_ret"
            data[spread_ret_col] = data[ret_col1] - data[ret_col2]

        if vol_col1 in data.columns and vol_col2 in data.columns:
            spread_vol_col = f"Spread_{c1}_{c2}_volume"
            data[spread_vol_col] = data[vol_col1] - data[vol_col2]

    return data.fillna(0).copy()
data = generate_spreads(data)

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\2232519225.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[spread_ret_col] = data[ret_col1] - data[ret_col2]
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\2232519225.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[spread_vol_col] = data[vol_col1] - data[vol_col2]
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\2232519225.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

Calendar dummies

In [134]:
try:
    month_dummies = pd.get_dummies(data.index.month, prefix="month", dtype=float)
    month_dummies.set_index(data.index, inplace=True)
    day_dummies = pd.get_dummies(data.index.dayofweek, prefix="weekday", dtype=float)
    day_dummies.set_index(data.index, inplace=True)
    presidency_year_dummies = pd.get_dummies(data.index.year % 4, prefix='pres_year', dtype='float')
    presidency_year_dummies.set_index(data.index, inplace=True)
    
    data = data.join([month_dummies, day_dummies, presidency_year_dummies])
except:
    print('Error: probably already created dummies.')

Volatility / Price Features (From ChatGPT)

In [135]:
daily_vix_vol = data['^VIX_close'] / (100 * np.sqrt(252))
data['vix_implied_var'] = np.square(daily_vix_vol)
    # Measures the percentage change in the ^VIX over 1 week and 1 month
data['vix_mom_1w'] = data['^VIX_close'].pct_change(periods=5)
data['vix_mom_1m'] = data['^VIX_close'].pct_change(periods=21)

data[f'vix_trend_1m'] = rolling_mean(data[f'^VIX_close'], 21)
data[f'vix_trend_3m'] = rolling_mean(data[f'^VIX_close'], 63)
data[f'vix_trend_6m'] = rolling_mean(data[f'^VIX_close'], 126)

# Calculate how far the current VIX is from its trend (as a percentage)
data['vix_trend_dist'] = (data['^VIX_close'] - data['vix_trend_3m']) / data['vix_trend_3m']

In [136]:
EPS = 1e-12

# If you already have a rolling_mean helper, keep it. Otherwise:
def rolling_mean(s, w):
    return s.rolling(w, min_periods=max(2, int(w*0.6))).mean()

def ewma_vol(r, lam=0.94):
    # EWMA variance per RiskMetrics: sigma_t^2 = (1-lam)*r_{t-1}^2 + lam*sigma_{t-1}^2
    # Use pandas ewm for convenience
    return r.pow(2).ewm(alpha=(1-lam), adjust=False).mean().clip(lower=0)

def rolling_autocorr(x, lag=1, window=63):
    # Rolling autocorrelation of x at a given lag
    # For stability, require at least ~60% of window
    minp = max(10, int(window*0.6))
    x0 = x
    x1 = x.shift(lag)
    return x0.rolling(window, min_periods=minp).corr(x1)

def realized_quarticity(r, window=63):
    # 3-month robust quarticity proxy (if daily): sum r^4 * (n / 3) approximation
    # Here we simply provide rolling sum of r^4; scaling optional depending on use
    minp = max(10, int(window*0.6))
    return (r.pow(4)).rolling(window, min_periods=minp).sum()

def build_vol_features(data, prefix, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94, target='close'):
    """
    Expects:
      data[f'{prefix}_log_price'] (daily log price)
      data[f'{prefix}_log_volume'] (daily log volume)
    Produces a suite of volatility-centric features for that prefix.
    """
    
    lp = data[f"{prefix}_log_{target}"]
    lv = data.get(f"{prefix}_log_volume", None)

    # Daily log return
    r = lp.diff()  # already log-price, so diff = log-return

    # --- Realized volatility proxies ---
    data[f"{prefix}_rv_1m"]  = r.rolling(day_w, min_periods=int(day_w*0.6)).var().clip(lower=0)          # variance
    data[f"{prefix}_rv_3m"]  = r.rolling(qtr_w, min_periods=int(qtr_w*0.6)).var().clip(lower=0)
    data[f"{prefix}_rv_1y"]  = r.rolling(yr_w,  min_periods=int(yr_w*0.6)).var().clip(lower=0)
    data[f"{prefix}_absrv_1m"] = r.abs().rolling(day_w, min_periods=int(day_w*0.6)).mean()               # mean |r|
    data[f"{prefix}_absrv_3m"] = r.abs().rolling(qtr_w, min_periods=int(qtr_w*0.6)).mean()

    # EWMA volatility (RiskMetrics-style)
    data[f"{prefix}_ewma_var"] = ewma_vol(r, lam=ewma_lambda)
    data[f"{prefix}_ewma_vol"] = np.sqrt(data[f"{prefix}_ewma_var"])

    # Volatility-of-volatility (how fast vol is changing)
    data[f"{prefix}_vol_speed_1w"] = data[f"{prefix}_rv_3m"].diff(5)                                      # weekly change in 3m var
    data[f"{prefix}_vol_mom_1m"]   = data[f"{prefix}_rv_3m"] - data[f"{prefix}_rv_1m"]                    # 3m vs 1m
    data[f"{prefix}_vol_mom_1y"]   = data[f"{prefix}_rv_1y"] - data[f"{prefix}_rv_3m"]

    # Volatility clustering proxies
    data[f"{prefix}_acf_sqret_lag1_3m"] = rolling_autocorr(r.pow(2), lag=1, window=qtr_w)
    data[f"{prefix}_acf_absret_lag1_3m"] = rolling_autocorr(r.abs(), lag=1, window=qtr_w)

    # Leverage effect proxy (contemporaneous corr between return and next day's vol)
    # Negative returns often precede higher vol; we proxy with corr(r_t, |r|_{t+1})
    data[f"{prefix}_lev_proxy_3m"] = r.shift(1).rolling(qtr_w, min_periods=int(qtr_w*0.6)).corr(r.abs()).shift(1)

    # Quarticity (heavy tails proxy)
    data[f"{prefix}_quarticity_3m"] = realized_quarticity(r, window=qtr_w)

    # Ratio features (normalized vol levels)
    data[f"{prefix}_vol_ratio_1m_3m"] = (data[f"{prefix}_rv_1m"] / (data[f"{prefix}_rv_3m"] + EPS))
    data[f"{prefix}_vol_ratio_3m_1y"] = (data[f"{prefix}_rv_3m"] / (data[f"{prefix}_rv_1y"] + EPS))
    data[f"{prefix}_ewma_over_3m"]    = (data[f"{prefix}_ewma_var"] / (data[f"{prefix}_rv_3m"] + EPS))

    # Price–volatility relation: distance from trend as a stress proxy
    data[f"{prefix}_price_trend_3m"]  = rolling_mean(lp, qtr_w)
    data[f"{prefix}_price_trend_dist"] = lp - data[f"{prefix}_price_trend_3m"]
    # Volatility when far below trend often spikes; include interaction
    data[f"{prefix}_vol_x_trend_dist"] = data[f"{prefix}_rv_1m"] * data[f"{prefix}_price_trend_dist"]

    # Volume–volatility links (if volume available)
    if lv is not None:
        dv = lv.diff()  # log-volume change
        data[f"{prefix}_vlm_var_1m"] = dv.rolling(day_w, min_periods=int(day_w*0.6)).var().clip(lower=0)
        data[f"{prefix}_vlm_var_3m"] = dv.rolling(qtr_w, min_periods=int(qtr_w*0.6)).var().clip(lower=0)
        # Corr between |r| and volume changes (vol–volume clustering)
        data[f"{prefix}_corr_absr_dlv_3m"] = r.abs().rolling(qtr_w, min_periods=int(qtr_w*0.6)).corr(dv)
        # Volume surprise proxy: current vs 3m trend
        data[f"{prefix}_vlm_trend_3m"] = rolling_mean(lv, qtr_w)
        data[f"{prefix}_vlm_trend_dist"] = lv - data[f"{prefix}_vlm_trend_3m"]
        # Vol reacts to volume surprises
        data[f"{prefix}_vol_x_vlm_surprise"] = data[f"{prefix}_rv_1m"] * data[f"{prefix}_vlm_trend_dist"]

    # 2. Calculate the spread against a realized variance feature you already have
    # We'll use a 1-month realized variance (rv_1m) for the S&P 500 as an example.
    # Ensure you have 'S&P_rv_1m' or a similar column from your previous code.
    data['{prefix}_vrp_spread'] = data['vix_implied_var'] - data[f'{prefix}_rv_1m']



    return data

# ---- Apply to all price series ---- #### both close and close_ret
for item in log_price_keys:
    data = build_vol_features(data, index, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94)
    data = build_vol_features(data, item, day_w=21, qtr_w=63, yr_w=252, ewma_lambda=0.94, target='close_ret')

# ---- Cross-index spillover features (optional but useful) ----
# Differences/spreads in contemporaneous vol across indices capture contagion/regime moves
data["SPX_minus_NDX_vol_1m"] = data["S&P_rv_1m"] - data["NASDAQ_rv_1m"]
data["SPX_minus_DJIA_vol_1m"] = data["S&P_rv_1m"] - data["DJIA_rv_1m"]
data["NDX_minus_DJIA_vol_1m"] = data["NASDAQ_rv_1m"] - data["DJIA_rv_1m"]
data['ES_minus_NQ_vol_1m'] = data["ES_rv_1m"] - data["NQ_rv_1m"]

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3478429046.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f"{prefix}_vol_mom_1y"]   = data[f"{prefix}_rv_1y"] - data[f"{prefix}_rv_3m"]
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3478429046.py:57: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f"{prefix}_acf_sqret_lag1_3m"] = rolling_autocorr(r.pow(2), lag=1, window=qtr_w)
C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\3478429046.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually 

In [137]:
data = data.copy().fillna(method='ffill').fillna(0)

C:\Users\rjsyo\AppData\Local\Temp\ipykernel_32460\547140819.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.copy().fillna(method='ffill').fillna(0)


In [138]:
data.dropna(inplace=True)

In [139]:
data

^VIX_close  ^VIX_high   ^VIX_low  ^VIX_open  ^TNX_close  \
Date                                                                  
2003-12-24   16.660000  16.770000  16.240000  16.650000       4.187   
2003-12-26   17.450001  17.490000  16.629999  16.760000       4.148   
2003-12-29   17.090000  17.860001  17.070000  17.809999       4.230   
2003-12-30   17.680000  17.969999  17.200001  17.320000       4.279   
2003-12-31   18.309999  18.860001  17.410000  17.490000       4.257   
...                ...        ...        ...        ...         ...   
2025-09-29   16.120001  16.290001  15.740000  15.840000       4.141   
2025-09-30   16.280001  16.700001  16.020000  16.490000       4.148   
2025-10-01   16.290001  17.280001  15.980000  17.280001       4.106   
2025-10-02   16.629999  16.920000  15.930000  16.120001       4.088   
2025-10-03   16.650000  17.059999  16.200001  16.350000       4.119   

            ^TNX_high  ^TNX_low  ^TNX_open  ^IRX_close  ^IRX_high  ^IRX_low  \
Date                                                                          
2003-12-24      4.246     4.182      4.240       0.872      0.877     0.872   
2003-12-26      4.176     4.147      4.176       0.847      0.866     0.833   
2003-12-29      4.239     4.160      4.166       0.857      0.862     0.852   
2003-12-30      4.300     4.242      4.255       0.912      0.912     0.882   
2003-12-31      4.282     4.249      4.265       0.907      0.912     0.907   
...               ...       ...        ...         ...        ...       ...   
2025-09-29      4.168     4.135      4.150       3.863      3.865     3.848   
2025-09-30      4.152     4.106      4.133       3.845      3.853     3.840   
2025-10-01      4.129     4.089      4.108       3.845      3.845     3.835   
2025-10-02      4.127     4.083      4.106       3.848      3.850     3.838   
2025-10-03      4.119     4.081      4.083       3.858      3.860     3.843   

            ^IRX_open  DX-Y.NYB_close  DX-Y.NYB_high  DX-Y.NYB_low  \
Date                                                                 
2003-12-24      0.875       87.540001      87.970001     87.449997   
2003-12-26      0.833       87.629997      87.650002     87.430000   
2003-12-29      0.852       87.349998      87.720001     87.220001   
2003-12-30      0.882       87.000000      87.419998     86.900002   
2003-12-31      0.912       86.919998      87.199997     86.360001   
...               ...             ...            ...           ...   
2025-09-29      3.850       97.910004      98.180000     97.769997   
2025-09-30      3.850       97.769997      98.050003     97.629997   
2025-10-01      3.835       97.709999      97.879997     97.459999   
2025-10-02      3.838       97.849998      98.129997     97.519997   
2025-10-03      3.843       97.711998      97.949997     97.599998   

            DX-Y.NYB_open   S&P_div  NASDAQ_div  DJIA_div      CPI  fed_funds  \
Date                                                                            
2003-12-24      87.949997  0.028232    0.000461  0.033565  185.500       0.98   
2003-12-26      87.489998  0.028211    0.000462  0.033542  185.500       0.98   
2003-12-29      87.680000  0.027841    0.000454  0.033091  185.500       0.98   
2003-12-30      87.389999  0.027836    0.000453  0.033202  185.500       0.98   
2003-12-31      86.980003  0.027811    0.000454  0.033125  185.500       0.98   
...                   ...       ...         ...       ...      ...        ...   
2025-09-29      98.169998  0.013561    0.005867  0.017258  323.364       4.08   
2025-09-30      97.910004  0.013510    0.005851  0.017232  323.364       4.08   
2025-10-01      97.820000  0.013464    0.005823  0.017212  323.364       4.08   
2025-10-02      97.699997  0.013448    0.005800  0.017181  323.364       4.08   
2025-10-03      97.852997  0.013449    0.005824  0.017093  323.364       4.08   

            credit_spread  consumer_sentiment  inflation_expectation  \
Date                   

### Split data

Training/testing 80/20 split

In [140]:
import math

### Set target

In [141]:
linear_model_target = 'next_ret'
drift = data.copy()

In [142]:
for contract in contracts:
    drift[f'{contract}_{linear_model_target}'] = drift[f'{contract}_log_close_ret'].shift(-1)
drift.dropna(inplace=True)

In [143]:
def split_data(data, split=0.8, embargo=2):
    cutoff = math.floor(len(data)*split)
    training_data = data.iloc[:(cutoff - embargo)]
    testing_data = data.iloc[(cutoff+embargo):]
    return training_data.copy(), testing_data.copy()

In [144]:
drift_training_data, drift_testing_data = split_data(drift)

### Determine feature importance

In [145]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance

In [146]:
def set_targets_features(target, training_data, contracts=contracts, additional_exclusions=[], additional_unscalables=[]):
    # Make sure we only fit on training_data and explanatory variables
    targets = [f'{contract}_{target}' for contract in contracts]
    month_dummies = [f'month_{month}' for month in range(1,13)]
    day_dummies = [f'weekday_{day}' for day in range(5)]
    presidency_year_dummies = day_dummies = [f'pres_year_{year}' for year in range(4)]
    rollover_dummy = [f'{contract}_Rollover' for contract in contracts]
    dummies = month_dummies + day_dummies + presidency_year_dummies + rollover_dummy
    additional_exclusions.extend([f'{contract}_Episode'])


    columns_to_exclude = [] + additional_exclusions

    unscalable_features = dummies + additional_unscalables
    scalable_features = [column for column in training_data.columns if column not in targets and column not in unscalable_features and column not in columns_to_exclude]
    all_features = unscalable_features + scalable_features

    return targets, all_features, unscalable_features, scalable_features

In [147]:
drift_targets, drift_all_features, drift_unscale_features, drift_scale_features = set_targets_features(linear_model_target, drift_training_data)

Let's try using exp weight decay

In [148]:
def determine_feature_importance(training_data, testing_data, features, target, contracts=contracts, mode='permutation', permutation_threshold = 0, selector_threshold=75):
    pruned_features = {}

    for contract in contracts:
        y_train = training_data[f'{contract}_{target}']
        y_test = testing_data[f'{contract}_{target}']
        X_train = training_data[features]
        X_test = testing_data[features]

        # Fit random forest
        rf = RandomForestRegressor(
            n_estimators = 500,
            max_depth=None,
            random_state=42,
            n_jobs=-1
        )
        rf.fit(X_train, y_train)

        if mode == 'permutation':
            result = permutation_importance(
                rf, X_test, y_test, # Make sure on test data
                n_repeats = 30,
                random_state=42,
                n_jobs=-1,
            )

            importances = pd.Series(result.importances_mean, index=X_train.columns).sort_values(ascending=False)
            selected_features = importances[importances > permutation_threshold].index.tolist()

            pruned_features[contract] = {
                'train_reduced': X_train[selected_features],
                'test_reduced': X_test[selected_features],
                'selected_features': selected_features
            }


        if mode == 'random_forest':
            selector_threshold = np.percentile(rf.feature_importances_, selector_threshold)
            selector = SelectFromModel(rf, threshold=selector_threshold, prefit=True)
            selected_features = training_data[features].columns[selector.get_support()]

            pruned_features[contract] = {
                'train_reduced': selector.transform(X_train),
                'test_reduced': selector.transform(X_test),
                'selected_features': selected_features
            }

    return pruned_features

Pickling for saving compute expensive dicts

In [149]:
import os, pickle, gzip, bz2, lzma
from pathlib import Path
from typing import Callable, Any, Tuple

In [150]:
def _opener(path: Path, mode: str):
    ext = path.suffix.lower()
    if ext in ('.pkl', '.pickle'):
        return path.open(mode)
    if ext == '.gz':
        return gzip.open(path, mode)
    if ext == '.bz2':
        return bz2.open(path, mode)
    if ext in ('.xz', '.lzma'):
        return lzma.open(path, mode)
    raise ValueError(f"Unsupported extension: {ext}")

def get_or_build_results(
    path: str,
    build_fn: Callable[..., dict],
    *args, **kwargs
) -> Tuple[dict, str]:

    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)

    # 1) Try load
    try:
        with _opener(p, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:

        try:
            # 2) Build
            results = build_fn(*args, **kwargs)

            # 3) Save
            with _opener(p, 'wb') as f:
                pickle.dump(results, f, protocol=pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print(f'Exception occured: {e}')

    return results

In [151]:
# Random forest for now because it's faster
pruned_features = get_or_build_results('model_data/drift_features_rf.pkl.gz', determine_feature_importance, drift_training_data, drift_testing_data, drift_all_features, 'next_ret', mode='random_forest')

for contract in contracts:
    print(pruned_features[contract]['selected_features'])

KeyboardInterrupt: 

### Normalize inputs

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [ ]:
def scale_features(training_data, testing_data, features):
    scaler = RobustScaler()
    scaler.fit(training_data[features]) # Fitting on training data

    train_scaled = training_data.copy()
    test_scaled = testing_data.copy()

    # Scale
    train_scaled[features] = scaler.transform(training_data[features])
    test_scaled[features] = scaler.transform(testing_data[features])

    # Save info on standardization for later
    #scaler_mu = pd.Series(scaler.mean_, index=features)
    #scaler_std = pd.Series(scaler.scale_, index=features)
    return train_scaled, test_scaled #, scaler_mu, scaler_std

In [ ]:
drift_train_scaled, drift_test_scaled = scale_features(drift_training_data, drift_testing_data, drift_scale_features)

### Linear Regression

In [ ]:
import warnings
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.decomposition import PCA
from sklearn.exceptions import ConvergenceWarning
from sklearn.cross_decomposition import PLSRegression
from mlxtend.evaluate import GroupTimeSeriesSplit

We are going to test with multiple different linear models to account for collinearity

In [ ]:
# Choose which models we want to use
linear_models = ['enet'] 

In [ ]:
# Function to get print results from the models
def eval_and_report(y_true, y_pred, model_name):
    print(f"{model_name:18s} | R^2: {r2_score(y_true, y_pred):.4f} | RMSE: {root_mean_squared_error(y_true, y_pred):.6f}")

Training function

In [ ]:
def train_models(train_scaled, test_scaled, target, contracts=contracts, pruned_features=pruned_features, models=linear_models):
    # Suppress all convergence warnings
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    results = {}

    for contract in contracts:
        results[contract] = {}
        features = pruned_features[contract]['selected_features'][:200]

        print(f"\n=== Target: {f'{contract}_{target}'} ===")
        X_train = train_scaled[features].copy()
        y_train = train_scaled[f'{contract}_{target}'].copy()
        X_test = test_scaled[features].copy()
        y_test = test_scaled[f'{contract}_{target}'].copy()

        # Create time series splits that isolate each contract
        ep_train = train_scaled[f'{contract}_Episode']
        gts = GroupTimeSeriesSplit(n_splits = ep_train.nunique()-2, test_size=1, window_type='expanding', gap_size=1)
        
        tscv = list(gts.split(
            X = X_train,
            y = y_train,
            groups=ep_train.values
        ))

        # 1. Ordinary Least Squares (OLS)
        if 'ols' in models:
            ols = LinearRegression()
            ols.fit(X_train, y_train)
            yhat_ols = ols.predict(X_test)
            eval_and_report(y_test, yhat_ols, "OLS")

            # Print top coefficients
            ols_coef = pd.Series(ols.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top OLS coeffs:\n", ols_coef.head(10))

            results[contract].update({
                "ols_model": ols,
                "ols_coefs": ols_coef,
                "train_data_ols": pd.Series(ols.predict(X_train), index=y_train.index, name=f"ols_train"),
                "yhat_ols": pd.Series(yhat_ols, index=y_test.index, name=f"{contract}_{target}_ols_pred"),
            })

        # 2. Ridge with CV over alphas (time-series CV)
        if 'ridge' in models:
            alphas = np.logspace(-4, 6, 100)

            ridge = RidgeCV(alphas=alphas, cv=tscv, fit_intercept=True)
            ridge.fit(X_train, y_train)
            yhat_ridge = ridge.predict(X_test)
            eval_and_report(y_test, yhat_ridge, f"Ridge (alpha={ridge.alpha_:.4g})")

            # Print top coefficients
            ridge_coef = pd.Series(ridge.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top Ridge coeffs:\n", ridge_coef.head(10))
            
            results[contract].update({
                "ridge_model": ridge,
                "ridge_coefs": ridge_coef,
                "train_data_ridge": pd.Series(ridge.predict(X_train), index=y_train.index, name=f"ridge_train"),
                "yhat_ridge": pd.Series(yhat_ridge, index=y_test.index, name=f"{contract}_{target}_ridge_pred"),
            })

        # 3. Lasso with CV over alphas (time-series CV)
        if 'lasso' in models and not 'lasso_ridge' in models:
            alphas = np.logspace(-4, 6, 100)

            lasso = LassoCV(alphas=alphas, cv=tscv, fit_intercept=True, n_jobs=-1)
            lasso.fit(X_train, y_train)
            yhat_lasso = lasso.predict(X_test)
            eval_and_report(y_test, yhat_lasso, f"Lasso (alpha={lasso.alpha_:.4g})")

            # Print top coefficients
            lasso_coef = pd.Series(lasso.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top Lasso coeffs:\n", lasso_coef.head(10))

            results[contract].update({
                "lasso_model": lasso,
                "lasso_coefs": lasso_coef,
                "train_data_lasso": pd.Series(lasso.predict(X_train), index=y_train.index, name=f"lasso_train"),
                "yhat_lasso": pd.Series(yhat_lasso, index=y_test.index, name=f"{contract}_{target}_lasso_pred"),
            })

        # 4. ElasticNet with CV over alphas and l1_ratios (time-series CV)
        if 'enet' in models:
            alphas = np.logspace(-4, 6, 30)
            l1_ratios = np.arange(0.1, 1, .1)   # 1.0 == Lasso, 0.0 == Ridge

            enet = ElasticNetCV(
                alphas=alphas,
                l1_ratio=l1_ratios,
                cv=tscv,
                fit_intercept=True,
                max_iter=5000,
                n_jobs = -1,
            )
            enet.fit(X_train, y_train)

            yhat_enet = enet.predict(X_test)
            eval_and_report(y_test, yhat_enet, f"ElasticNet (alpha={enet.alpha_:.4g}, l1_ratio={enet.l1_ratio_})")

            # Print top coefficients
            enet_coef = pd.Series(enet.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top ElasticNet coeffs:\n", enet_coef.head(10))

            results[contract].update({
                "enet_model": enet,
                "enet_coegs": enet_coef,
                "train_data_enet": pd.Series(enet.predict(X_train), index=y_train.index, name=f"enet_train"),
                "yhat_enet": pd.Series(yhat_enet, index=y_test.index, name=f"{contract}_{target}_enet_pred"),
            })

        # 5. PCA on OLS
        if 'pca' in models:
            pca = PCA(n_components=.9).fit(X_train) # keep x% of variance and fit to training set
            train_pca = pca.transform(X_train)
            test_pca = pca.transform(X_test)

            ols_pca = LinearRegression()
            ols_pca.fit(train_pca, y_train)
            yhat_pca = ols_pca.predict(test_pca)
            eval_and_report(y_test, yhat_pca, "OLS+PCA")

            results[contract].update({
                "pca_model": ols_pca,
                "train_data_pca": pd.Series(ols_pca.predict(train_pca), index=y_train.index, name=f"pca_train"),
                "yhat_pca": pd.Series(yhat_pca, index=y_test.index, name=f"{contract}_{target}_pca_pred")
            })

        # 6. Use lasso into ridge
        if 'lasso_ridge' in models:
            alphas = np.logspace(-4, 6, 100)

            lasso = LassoCV(alphas=alphas, cv=tscv, fit_intercept=True)
            lasso.fit(X_train, y_train)
            yhat_lasso = lasso.predict(X_test)
            eval_and_report(y_test, yhat_lasso, f"Lasso (alpha={lasso.alpha_:.4g})")

            # Print top coefficients
            lasso_coef = pd.Series(lasso.coef_, index=features).sort_values(key=np.abs, ascending=False)
            print("Top Lasso coeffs:\n", lasso_coef.head(10))

            results[contract].update({
                "lasso_model": lasso,
                "lasso_coefs": lasso_coef,
                "train_data_lasso": pd.Series(lasso.predict(X_train), index=y_train.index, name=f"lasso_train"),
                "yhat_lasso": pd.Series(yhat_lasso, index=y_test.index, name=f"{contract}_{target}_lasso_pred"),
            })
        
            # Get columns where coefficients are nonzero
            important_features = X_train.columns[np.abs(lasso.coef_) > 0]
            X_train_reduced = X_train[important_features]
            X_test_reduced = X_test[important_features]

            alphas = np.logspace(-4, 3, 30)

            lasso_ridge = RidgeCV(alphas=alphas, cv=tscv, fit_intercept=True)
            lasso_ridge.fit(X_train_reduced, y_train)
            yhat_lasso_ridge = lasso_ridge.predict(X_test_reduced)
            eval_and_report(y_test, yhat_lasso_ridge, f"Lasso_Ridge (alpha={lasso_ridge.alpha_:.4g})")

            # Print top coefficients
            lasso_ridge_coef = pd.Series(lasso_ridge.coef_, index=important_features).sort_values(key=np.abs, ascending=False)
            print("Top Lasso_Ridge coeffs:\n", lasso_ridge_coef.head(10))

            results[contract].update({
                "lasso_ridge_model": lasso_ridge,
                "lasso_ridge_coefs": lasso_ridge_coef,
                "train_data_lasso_ridge": pd.Series(lasso_ridge.predict(X_train_reduced), index=y_train.index, name=f"lasso_ridge_train"),
                "yhat_lasso_ridge": pd.Series(yhat_lasso_ridge, index=y_test.index, name=f"{contract}_{target}_lasso_ridge_pred"),
            })

        # 7. Partial Least Squares (PLS) Regression
        if 'pls' in models:
            # First, find the best number of components using cross-validation
            best_n = -1
            best_score = -np.inf
            
            # Test a range of components
            for n in range(1, 8):
                pls_cv = PLSRegression(n_components=n)
                # Use a time-series friendly CV
                scores = cross_val_score(pls_cv, X_train, y_train, cv=tscv, scoring='r2')
                
                if np.mean(scores) > best_score:
                    best_score = np.mean(scores)
                    best_n = n

            print(f"PLS best n_components: {best_n}")

            # Now, fit the final PLS model with the optimal number of components
            pls = PLSRegression(n_components=best_n)
            pls.fit(X_train, y_train)
            yhat_pls = pls.predict(X_test)
            eval_and_report(y_test, yhat_pls, f"PLS (n={best_n})")

            # can get coefficients if needed, but they are in PLS component space
            # pls_coef = pd.Series(pls.coef_, index=features)

            # Store results
            results[contract].update({
                "pls_model": pls,
                # "pls_coefs": pls_coef,
                "train_data_pls": pd.Series(pls.predict(X_train).ravel(), index=y_train.index, name=f"pls_train"),
                "yhat_pls": pd.Series(yhat_pls.ravel(), index=y_test.index, name=f"{contract}_{target}_pls_pred"),
            })
            


    return results

In [ ]:
drift_results = get_or_build_results('model_data/drift_training_results.pkl.gz', train_models, drift_train_scaled, drift_test_scaled, linear_model_target, models=linear_models)

TypeError: _frombuffer() takes 4 positional arguments but 5 were given

Make into new df

In [ ]:
def df_from_predictions(data, target, results, contracts=contracts, models=linear_models):
    df = data.copy()

    for contract in contracts:
        for model in models:
            df[f'{contract}_{target}_pred_{model}'] = pd.concat([results[contract][f'train_data_{model}'],results[contract][f'yhat_{model}']])
    df.dropna(inplace=True)

    return df


In [ ]:
drift = df_from_predictions(drift, linear_model_target, drift_results)

In [ ]:
drift

Plot these results

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
def plot_results(data, baseline, measures, title, subtitles, contracts=contracts, models=linear_models):
    fig = make_subplots(
        rows=len(contracts), cols=1, shared_xaxes=True, vertical_spacing=0.06,
        subplot_titles=[contract for contract in contracts]
    )

    for i, (contract) in enumerate(contracts, start=1):
        show_leg = (i == 1)

        # True next_ret
        fig.add_trace(
            go.Scatter(x=data.index, y=data[f'{contract}_{baseline}'], name=f'{contract}_{baseline}',
                    mode="lines", line=dict(width=1.6),
                    showlegend=show_leg, legendgroup="true"),
            row=i, col=1
        )

        # Plot predictions
        for model in models:
            for measure in measures:
                fig.add_trace(
                    go.Scatter(x=data.index, y=data[f'{contract}_{measure}_{model}'], name=f"{model} prediction",
                            mode="lines", line=dict(width=1.4, dash="dot"),
                            showlegend=show_leg, legendgroup=f"{model}"),
                    row=i, col=1
                )

    fig.update_layout(
        title=title,
        height=2500,
        hovermode="x unified",
        template="plotly_white",
        margin=dict(t=80, r=30, b=80, l=70),
        legend=dict(orientation="h", yanchor="top", y=-0.12, xanchor="left", x=0)
    )

    for r in range(1, len(contracts) + 1):
        fig.update_yaxes(title_text=subtitles, row=r, col=1)

    fig.show()

In [ ]:
plot_results(drift, 'log_close_ret', [f'{linear_model_target}_pred'], 'Linear Regression on Drift', 'Log Return', contracts=contracts[:12])
plot_results(drift, 'log_close_ret', [f'{linear_model_target}_pred'], 'Linear Regression on Drift', 'Log Return', contracts=contracts[12:])

Overall returns

In [ ]:
def build_price_path(df, pred, linear_model_target=linear_model_target, contracts=contracts, models=linear_models):
    df = df.copy()

    for contract in contracts:
        for model in models:
            df[f'{contract}_{pred}_pred_{model}'] = np.exp(df[f'{contract}_log_close'].iloc[0]) * np.exp(df[f'{contract}_{linear_model_target}_pred_{model}'].cumsum())

            df[f'{contract}_close'] = np.exp(df[f'{contract}_log_close'].iloc[0]) * np.exp(df[f'{contract}_log_close_ret'].cumsum())

    return df

In [ ]:
plot_drift = build_price_path(drift, 'price')
plot_results(plot_drift, 'close', ['price_pred'], 'Price Path Evolution of Drift', 'Price', contracts=contracts[:12])
plot_results(plot_drift, 'close', ['price_pred'], 'Price Path Evolution of Drift', 'Price', contracts=contracts[12:])

# GARCH Volatility Model

In [ ]:
from arch import arch_model

In [ ]:
# Squared residuals from previous -- using ElasticNet
linear_drift_model = 'enet'

Set up some GARCH models, check their fit our data

In [ ]:
garch_data = drift.copy()
garch_scale = 100


# Get drift residuals:
for contract in contracts:
    garch_data[f'{contract}_drift_residuals_{linear_drift_model}'] = drift[f'{contract}_{linear_model_target}'] - drift[f'{contract}_{linear_model_target}_pred_{linear_drift_model}'] # Calculate residual from enet
    garch_data[f'{contract}_drift_residuals_{linear_drift_model}'] = garch_data[f'{contract}_drift_residuals_{linear_drift_model}'] * garch_scale # Scaling for better convergence

# Split data
garch_training, garch_testing = split_data(garch_data)

# Models, distributions, and paramters to check

garch_models = {
    'Garch': 'Garch',
    'Egarch': 'Egarch',
    'GJR_Garch': 'Garch',
    'APARCH': 'APARCH'
}

dists = ['normal', 't', 'skewt']

params = {
    'Garch': {'p': 1, 'o': 0, 'q': 1},
    'Egarch': {'p': 2, 'o': 1, 'q': 1},
    'GJR_Garch': {'p': 1, 'o': 1, 'q': 1},
    'APARCH': {'p': 1, 'o': 1, 'q': 1}
}

In [ ]:
garch_test_models = ['APARCH']

## Forecast our testing period with the chosen models

In [ ]:
refit_freq = 21 # Frequency to refit at -- slightly larger helps with signal:noislambda, but not too big

def forecast_garch(garch_data, contracts=contracts, models=garch_test_models, linear_drift_model=linear_drift_model):
    garch_training, garch_testing = split_data(garch_data)

    for contract in contracts:
        for model in models:
            drift_residuals = garch_data[f'{contract}_drift_residuals_{linear_drift_model}']

            scaled_predictions = []
            for i in range(len(garch_training), len(garch_training) + len(garch_testing)): # Walk forward, forecast each step and refit model based on new inputs

                # Ensure we fit on first day and at our frequency
                testing_day = i - len(garch_training)
                is_refit_day = testing_day % refit_freq == 0
                
                if is_refit_day:
                    garch = arch_model(drift_residuals.iloc[:i], vol=garch_models[model], **params[model], mean='Zero', dist='skewt')
                    garch_fit = garch.fit(disp='off') # fit

                forecast = garch_fit.forecast(horizon=1)

                pred_variance = forecast.variance.iloc[-1].values[0]
                scaled_predictions.append(pred_variance)

            # Get exponentiation of std
            if model == 'APARCH':
                delta = garch_fit.params['delta']
            else:
                delta = 2
            
            unscaled_predictions = pd.Series(scaled_predictions, index=garch_testing.index) / (garch_scale ** delta) # Unscale (inputs scaled by 100 for convergence, var is squared, but APARCH may return delta =/= 2)
            final_predictions = unscaled_predictions ** (1 / delta)

            garch_testing[f'{contract}_std_pred_{model}'] = final_predictions

    return garch_testing.to_dict()

In [ ]:
garch_testing = pd.DataFrame(get_or_build_results('model_data/garch_testing_df.pkl.gz', forecast_garch, garch_data))
garch_testing

In [ ]:
predictions = drift.copy()

Let's add a rolling IC column to test with damping mu projections later

In [ ]:
for contract in contracts:
    predictions[f'{contract}_{linear_model_target}'] = predictions[f'{contract}_log_close'].diff().shift(-1).dropna(inplace=True)
    for model in linear_models:
        predictions[f'{contract}_IC_{model}'] = predictions[f'{contract}_{linear_model_target}'].rolling(window=21).corr(predictions[f'{contract}_{linear_model_target}_pred_{model}'])

Convert to variance / std

In [ ]:
for contract in contracts:
    for model in garch_test_models:
        predictions[f'{contract}_std_pred_{model}'] = garch_testing[f'{contract}_std_pred_{model}']

# Backtesting

In [ ]:
from scipy.stats import norm # For confidence scaling testing

Sizing strategy 1: Using Merton portfolio optimization

In [ ]:
def kelly_fraction(mu, rf, sigma, fraction=.5):
    size = fraction * (mu - rf) / (sigma**2 + 1e-8) 
    return size

Returns info

In [ ]:
def sharpe_and_mdd(df, col_equity, col_rf="ten_yr", periods_per_year=252):
    eq = df[col_equity]
    rf = df[col_rf]

    # compute portfolio simple returns
    port_rets = eq.pct_change().dropna()
    # align rf
    rf_aligned = rf.reindex(port_rets.index).astype(float)

    # excess returns (assuming rf is already per-period, e.g. daily)
    excess = port_rets - ((1+rf_aligned/100)**(1/periods_per_year)-1)
    sharpe = (excess.mean() / excess.std()) * np.sqrt(periods_per_year) if excess.std() > 0 else np.nan

    # max drawdown
    running_max = eq.cummax()
    drawdown = eq / running_max - 1.0
    max_dd = drawdown.min()

    return sharpe, max_dd

## Trade based on our models

### Backtest

In [ ]:
bt_models = ['APARCH'] # Volatility models

Get historical margin amounts

In [ ]:
historical_margins = {}

for contract in contracts:
    historical_margins[contract] = pd.read_csv(f'margins_data/{contract}_Historical_Margin.csv')
    historical_margins[contract]['Date'] = pd.to_datetime(historical_margins[contract]['Date'])
    historical_margins[contract].set_index(historical_margins[contract]['Date'], inplace=True)
    historical_margins[contract].drop(columns=['Date'], inplace=True)
    historical_margins[contract].rename(columns={'Long Margin': f'{contract}_Long', 'Short Margin': f'{contract}_Short'}, inplace=True)

In [ ]:
# Set initial conditions
starting_cash = 500_000
commission = 2.5
drawdown_lock = .2

initial_margin = 1.1 # 10% extra assumption on initial entry

tick_size = {
    'ES': 0.25, 
    'NQ': 0.25, 
    'YM': 1.00,
    'FF': 0.0025, 
    'TU': 0.00390625, 
    'FV': 0.0078125, 
    'TY': 0.015625, 
    'US': 0.03125, 
    'SI': 0.005,
    'HG': 0.0005, 
    'PL': 0.10, 
    'GC': 0.10, 
    'CL': 0.01, 
    'NG': 0.001,
    'JY': 0.0000005,
    'BP': 0.0001, 
    'EU': 0.00005,
    'CD': 0.00005,
    'C': 0.0025, 
    'W': 0.0025, 
    'S': 0.0025, 
    'LE': 0.00025, 
    'HE': 0.00025, 
}

tick_value = {
    'ES': 12.50, 
    'NQ': 5.00, 
    'YM': 5.00,
    'FF': 10.4175, 
    'TU': 7.8125, 
    'FV': 7.8125, 
    'TY': 15.625, 
    'US': 31.25, 
    'GC': 10.00, 
    'SI': 25.00,
    'HG': 12.50, 
    'PL': 5.00, 
    'CL': 10.00, 
    'NG': 10.00,
    'JY': 6.25,
    'BP': 6.25, 
    'EU': 6.25,
    'CD': 5.00,
    'C': 12.50, 
    'W': 12.50, 
    'S': 12.50, 
    'LE': 10.00, 
    'HE': 10.00, 
}

multipliers = np.array([tick_value[contract]/tick_size[contract] for contract in contracts])

In [ ]:
def plot_bt(data, baseline, measures, title, subtitles, models=bt_models):
    fig = make_subplots(
        rows=len(models), cols=1, shared_xaxes=True, vertical_spacing=0.06,
        subplot_titles=[model for model in models]
    )

    for i, (model) in enumerate(models, start=1):
        show_leg = (i == 1)

        # Baseline
        fig.add_trace(
            go.Scatter(x=data.index, y=data[f'{baseline}_{model}'], name=f'{baseline}_{model}',
                    mode="lines", line=dict(width=1.6),
                    showlegend=show_leg, legendgroup="true"),
            row=i, col=1
        )

        # Plot predictions
        for measure in measures:
            fig.add_trace(
                go.Scatter(x=data.index, y=data[f'{measure}_{model}'], name=f"{model} prediction",
                        mode="lines", line=dict(width=1.4, dash="dot"),
                        showlegend=show_leg, legendgroup=f"{measure}"),
                row=i, col=1
            )

    fig.update_layout(
        title=title,
        height=400,
        hovermode="x unified",
        template="plotly_white",
        margin=dict(t=80, r=30, b=80, l=70),
        legend=dict(orientation="h", yanchor="top", y=-0.12, xanchor="left", x=0)
    )

    for r in range(1, len(contracts) + 1):
        fig.update_yaxes(title_text=subtitles, row=r, col=1)

    fig.show()

Historical  -- Multiple different windows including static, Ledoit-Wolf, and rolling to test

In [191]:
from sklearn.covariance import LedoitWolf

In [192]:
def lw_correlation(data, corr_column='log_close_ret', contracts=contracts):
    # Get returns
    X = data[[f'{contract}_{corr_column}' for contract in contracts]]

    # Fit LW
    LW = LedoitWolf().fit(X.values)

    # Get cov matrix
    Sigma = LW.covariance_

    # Convert to correlation
    std = np.sqrt(np.clip(np.diag(Sigma), 1e-18, None))
    D_inv = np.diag(1.0 / std) # Since this is diagonal, inv is easy
    R = D_inv @ Sigma @ D_inv
    R = 0.5 * (R + R.T) # Enforce symmetry
    np.fill_diagonal(R, 1.0) # Enforce unit diagonal

    return R

In [ ]:
# Calculate historical correlation matrix
bt = predictions.copy()
bt_train, bt_test = split_data(bt)

contract_corr_train = []
for i in range(len(bt_train), len(bt_train) + len(bt_test)):
    contract_corr_train.append(bt_train[[f'{contract}_log_close_ret' for contract in contracts]].corr()) # This may be inefficient, but simplest way I can think of for right now to interface with the other windows

contract_corr_rolling = []
for i in range(len(bt_train), len(bt_train) + len(bt_test)):
    contract_corr_rolling.append(bt.iloc[:i][[f'{contract}_log_close_ret' for contract in contracts]].corr())

contract_corr_train_LW = []
for i in range(len(bt_train), len(bt_train) + len(bt_test)):
    contract_corr_train_LW.append(lw_correlation(data))

contract_corr_LW_rolling = []
for i in range(len(bt_train), len(bt_train) + len(bt_test)):
    contract_corr_LW_rolling.append(lw_correlation(data[:i]))


def rolling_corr(data, start_i, end_i, window, LW=False):
    out = []
    for i in range(start_i, end_i):
        j = max(0, i-window)

        if not LW:
            out.append(data.iloc[j:i].corr())
        else:
            out.append(lw_correlation(data.iloc[j:i]))
    return out

test_start = len(bt_train)
test_end = len(bt_train) + len(bt_test)

contract_corr_rolling_21 = rolling_corr(bt, test_start, test_end, 21)
contract_corr_rolling_126 = rolling_corr(bt, test_start, test_end, 126)
contract_corr_rolling_252 = rolling_corr(bt, test_start, test_end, 252)

contract_corr_rolling_21_LW = rolling_corr(bt, test_start, test_end, 21, LW=True)
contract_corr_rolling_126_LW = rolling_corr(bt, test_start, test_end, 126, LW=True)
contract_corr_rolling_252_LW = rolling_corr(bt, test_start, test_end, 252, LW=True)

In [ ]:
contract_corr_windows = {
    'Train': contract_corr_train,
    'Rolling': contract_corr_rolling, 
    '21': contract_corr_rolling_21, 
    '126': contract_corr_rolling_126, 
    '252': contract_corr_rolling_252,
    'Train_LW': contract_corr_train_LW,
    'Rolling_LW': contract_corr_LW_rolling,
    '21_LW': contract_corr_rolling_21_LW,
    '126_LW': contract_corr_rolling_126_LW,
    '252_LW': contract_corr_rolling_252_LW
}

In [ ]:
def multi_asset_kelly_fraction(mu, sigma, fraction=.5):
    weights = fraction * np.linalg.solve(sigma, mu) 
    return weights

In [ ]:
from tqdm import tqdm

Backtesting on multiple assets

In [ ]:
def multi_asset_backtest(max_leverage=12, kelly_frac=.4, bt_models=bt_models, contract_corr=contract_corr_train):
    # Create backtest df copy
    bt = predictions.copy()
    bt = bt.join([margin_data for margin_data in historical_margins.values()])
    bt = bt.replace(0, pd.NA)
    bt = bt.fillna(method='ffill')
    bt_train, bt_test = split_data(bt)
    bt_data = {}
    linear_drift_model = 'enet'

    T = len(bt_test)

    close_cols  = [f'{contract}_log_close' for contract in contracts]
    ret_cols   = [f'{contract}_log_close_ret' for contract in contracts]
    long_cols  = [f'{contract}_Long' for contract in contracts]
    short_cols = [f'{contract}_Short' for contract in contracts]
    mu_cols    = [f'{contract}_{linear_model_target}_pred_{linear_drift_model}' for contract in contracts]
    roll_cols  = [f'{contract}_Rollover' for contract in contracts]

    rf_mat = np.power((1+(bt_test['^TNX_close'].to_numpy(float))/100), 1/252)-1  # Daily
    raw_mu_mat = bt_test[mu_cols].to_numpy(float)
    ret_mat = np.exp(bt_test[ret_cols].to_numpy(float)) - 1
    close_mat = np.exp(bt_test[close_cols].to_numpy(float))
    maintL_mat = bt_test[long_cols].to_numpy(float)
    maintS_mat = bt_test[short_cols].to_numpy(float)
    rollday_mat = 1 - bt_test[roll_cols].shift(-1).fillna(0).to_numpy(float) # Set rollover days equal to 1 - rollover so we can multiply expected return and positions by this and set them equal to 0 day before

    for model in bt_models:
        bt_test[f'cash_{model}'] = 0.0
        bt_test[f'cash_{model}'].iloc[0] = starting_cash
        bt_test[f'margin_{model}'] = 0.0
        equal_weighted_returns = np.exp(bt_test[ret_cols].cumsum())
        portfolio_values = equal_weighted_returns * (starting_cash / len(contracts))
        bt_test[f'baseline_wealth_{model}'] = portfolio_values.sum(axis=1)
        bt_test[f'margin_usage_{model}'] = 0
        
        vol_cols = [f'{contract}_std_pred_{model}' for contract in contracts]
        pred_vol_mat = bt_test[vol_cols].to_numpy(float)
        pred_var_mat = np.square(pred_vol_mat)

        margins = np.full(len(contracts),0.0)
        positions = np.full(len(contracts),0)
        cash = starting_cash
        last_total_equity = starting_cash
        total_equity = starting_cash

        # Metrics
        positions_mat = np.full(len(contracts), 0)
        margins_mat = np.full(len(contracts), 0.0)
        profit_mat = np.full(len(contracts), 0.0)
        total_commission = 0.0

        # Loop
        for t in tqdm(range(1, T)):
            rf = rf_mat[t]
            lock_up = False

            # Get predicted variances for next day and create covariance matrix based on historical correlation
            pred_vol = pred_vol_mat[t]
            D = np.diag(pred_vol)
            pred_cov = D @ contract_corr[t-1] @ D

            # mu needs to be adjusted upwards for sigma
            raw_mu = raw_mu_mat[t]
            if model not in garch_models.keys():
                mu = np.exp(raw_mu + (1/2) * pred_var_mat[t]) - 1
            else:
                # We need a different adjustment for our garch models since they are not based on a normality assumption
                garch_drift_adjustment = (1/2) * pred_var_mat[t] - (1/6) * np.power(pred_vol_mat[t], 3)
                mu = np.exp(raw_mu + garch_drift_adjustment) - 1

            # Check if rollover day tomorrow
            mu = mu * rollday_mat[t]

            # Update margin value
            prevClose = close_mat[t-1]
            close = close_mat[t]
            ret = ret_mat[t]
            delta = ret*prevClose # This calculation is basically assuming we are making our trading decisions at the end of every day
            pnl_today = positions * delta * multipliers
            margins += pnl_today

            # Calculate total equity for lockup criteria
            last_total_equity = total_equity
            total_equity = margins.sum() + cash

            # If drawdown criteria met, lock portfolio for day
            lock_up = total_equity/(last_total_equity + 1e-8) <= (1 - drawdown_lock)

            # Check maintenance requirements
            maint_long = maintL_mat[t]
            maint_short = maintS_mat[t]
            maint_side = np.where(positions > 0, maint_long, maint_short)

            maint_diff = np.abs(maint_side*positions) - margins

            if maint_diff.sum() > cash:
                # Liquidate -- simple, not greedy (may be less effective)
                cash += margins.sum()
                margins *= 0
                positions *= 0
            else:
                # Top ups & off load to cash -- notice that 'negative' maint_diff means return to normal maintenance and add to cash
                margins += maint_diff
                cash -= maint_diff.sum()

            # Recalculate total equity
            total_equity = margins.sum() + cash

            # Make trades
            if not lock_up:
                # Get bet sizing with fractional kelly -- default is .5
                target_notionals = np.clip(multi_asset_kelly_fraction(mu-rf, pred_cov, fraction=kelly_frac), -max_leverage, max_leverage) * total_equity
                maint_side = np.where(target_notionals > 0, maint_long, maint_short)

                # Portfolio allocation
                position_found = False
                k = 1 # Scaling constant to reduce portfolio investment if needed
                while not position_found and k > 0:
                    notional_per_contract = close * multipliers
                    initial_costs = maint_side*initial_margin # Get current maintenance * initial margin percentage requirement
                    target_positions = target_notionals * k / notional_per_contract # Get number of contracts to calculate margin investment, ignoring commission
                    target_positions = np.where(target_positions > 0, np.floor(target_positions), np.ceil(target_positions))
                    
                    target_positions = target_positions * rollday_mat[t] # Make sure we close positions that are rolling tomorrow
                    investments = np.abs(target_positions)*initial_costs - margins
                    required_margin = investments.sum()

                    if required_margin > cash:
                        k -= .05 # Find min amount we can invest by simple scale down
                    else:
                        position_found = True
            else:
                investments = np.full(len(contracts),0.0)
                required_margin = 0
                target_positions = positions

            # Update cash
            position_change = np.abs(positions - target_positions).sum()
            commission_charge = position_change * commission
            cash -= required_margin
            cash -= commission_charge # Commission per trade
            total_commission += commission_charge
            bt_test.iloc[t, bt_test.columns.get_loc(f'cash_{model}')] = cash

            # Update positions and margins
            positions = target_positions
            margins += investments
            bt_test.iloc[t, bt_test.columns.get_loc(f'margin_{model}')] = margins.sum()

            positions_mat = np.vstack((positions_mat, positions))
            margins_mat = np.vstack((margins_mat, margins))
            profit_mat = np.vstack((profit_mat, pnl_today))
        
        # Update models in bt_test
        total_value = f'total_value_{model}'
        bt_test[total_value] = bt_test[f'margin_{model}'] + bt_test[f'cash_{model}']
        bt_data[model] = {'positions': positions_mat, 'margins': margins_mat, 'profit': profit_mat, 'total_commission': total_commission}

    return bt_test, bt_data

In [ ]:
bt_test, bt_data = multi_asset_backtest(max_leverage=13, kelly_frac=.3, contract_corr=contract_corr_windows['252'])

In [ ]:
for model in bt_models:
    print(f'=== {model} ===')
    total_value = f'baseline_wealth_{model}'
    
    final_training_value = bt_test[total_value].iloc[-1]
    training_cagr = 100 * ((final_training_value/bt_test[total_value].iloc[0])**(1/(bt_test.index[-1].year - bt_test.index[0].year + bt_test.index[-1].month/12))-1)
    print(f'Final value for {model}, baseline: {final_training_value:.2f}')
    
    test_sharpe, test_mdd = sharpe_and_mdd(bt_test, total_value, col_rf="^TNX_close")
    print(f"Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}, CAGR: {training_cagr:.2f}%\n")

    total_value = f'total_value_{model}'

    final_testing_value = bt_test[total_value].iloc[-1]
    testing_cagr = 100*((final_testing_value/bt_test[total_value].iloc[0])**(1/(bt_test.index[-1].year - bt_test.index[0].year + bt_test.index[-1].month/12))-1)
    print(f'Final value for {model}: {final_testing_value:.2f}')
    
    test_sharpe, test_mdd = sharpe_and_mdd(bt_test, total_value, col_rf="^TNX_close")
    print(f"Sharpe: {test_sharpe:.2f}, Max Drawdown: {test_mdd:.2%}, CAGR: {testing_cagr:.2f}%")
    print(f'Total commission: {bt_data[model]['total_commission']}')

    average_margin_util = bt_test[f'margin_usage_{model}'].sum() / len(bt_test)
    print(f"Average Margin Utilizization: {average_margin_util*100:.2f}%")
    print('\n')

    

In [ ]:
plot_bt(bt_test, 'baseline_wealth', ['total_value'], 'Backtest Performance', 'Wealth', models=bt_models)

In [ ]:
def plot_margin_positions_profit(bt_test, bt_data, model='APARCH', contracts=contracts, title=None, step_positions=True):
    # --- Build DataFrames from matrices ---
    T, N = bt_data['positions'].shape
    idx = bt_test.index[-T:] if len(bt_test) >= T else bt_test.index  # align end if there's a warmup
    
    pos_df = pd.DataFrame(bt_data['positions'], index=idx, columns=contracts)
    mar_df = pd.DataFrame(bt_data['margins'],   index=idx, columns=contracts)
    pnl_df = pd.DataFrame(bt_data['profit'],    index=idx, columns=contracts)

    total_equity = bt_test.loc[idx, f'total_value_{model}'].astype(float)
    total_margin = bt_test.loc[idx, f'margin_{model}'].astype(float)
    safe_equity  = total_equity.replace(0, np.nan)

    # Per-contract margin % of equity
    mar_pct_df = mar_df.div(safe_equity, axis=0)
    margin_pct_total = (total_margin / safe_equity)

    # P&L aggregates
    daily_total_pnl = pnl_df.sum(axis=1)
    cum_total_pnl   = daily_total_pnl.cumsum()
    cum_pnl_df      = pnl_df.cumsum()  # if you ever want per-contract cum P&L

    # --- Figure layout ---
    fig = make_subplots(
        rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.06,
        subplot_titles=(
            f"{model}: Equity & Margin % of Equity",
            f"{model}: Per-Contract Margin Share (% of Equity)",
            f"{model}: Positions by Contract",
            f"{model}: P&L (Daily per-contract bars) & Cumulative Total"
        ),
        specs=[
            [{"secondary_y": True}],  # row 1: add secondary y for % axis
            [{}],
            [{}],
            [{}],
        ]
    )

    # 1) Equity & margin %
    fig.add_trace(
        go.Scatter(x=idx, y=total_equity, name="Total Equity", mode="lines",
                   line=dict(width=1.8)),
        row=1, col=1, secondary_y=False
    )
    fig.add_trace(
        go.Scatter(x=idx, y=(margin_pct_total * 100.0), name="Margin / Equity (%)",
                   mode="lines", line=dict(width=1.6, dash="dot")),
        row=1, col=1, secondary_y=True
    )
    fig.update_yaxes(title_text="Equity ($)", row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Margin / Equity (%)", row=1, col=1, secondary_y=True)

    # 2) Per-contract margin share (% of equity), stacked
    for c in contracts:
        fig.add_trace(
            go.Scatter(
                x=idx, y=(mar_pct_df[c] * 100.0),
                name=f"{c} margin%",
                mode="lines",
                stackgroup="mshare", groupnorm="percent"  # visually: share of margin vs equity
            ),
            row=2, col=1
        )
    fig.update_yaxes(title_text="Per-contract Margin (% equity)", row=2, col=1)

    # 3) Positions (contracts)
    for c in contracts:
        fig.add_trace(
            go.Scatter(
                x=idx, y=pos_df[c], name=f"{c} pos",
                mode="lines",
                line=dict(width=1.2),
                line_shape="hv" if step_positions else None
            ),
            row=3, col=1
        )
    fig.update_yaxes(title_text="Contracts", row=3, col=1)

    # 4) P&L: cumulative per-contract lines + cumulative total line
    cum_pnl_df    = pnl_df.cumsum()
    cum_total_pnl = cum_pnl_df.sum(axis=1)

    for c in contracts:
        fig.add_trace(
            go.Scatter(
                x=idx, y=cum_pnl_df[c], name=f"{c} cum P&L",
                mode="lines", line=dict(width=1.6)
            ),
            row=4, col=1
        )

    # Total cumulative P&L (thicker line)
    fig.add_trace(
        go.Scatter(
            x=idx, y=cum_total_pnl, name="Cumulative P&L (total)",
            mode="lines", line=dict(width=2.2, dash="solid")
        ),
        row=4, col=1
    )

    fig.update_yaxes(title_text="Cumulative P&L ($)", row=4, col=1)

    # --- Global layout ---
    fig.update_layout(
        title=title or f"Portfolio Margin, Positions & P&L — {model}",
        height=1100,
        hovermode="x unified",
        template="plotly_white",
        margin=dict(t=70, r=40, b=60, l=70),
        legend=dict(orientation="h", yanchor="top", y=-0.12, xanchor="left", x=0)
    )
    fig.show()

In [ ]:
testing_model = 'APARCH'
bt_test, bt_data = multi_asset_backtest(max_leverage=12, kelly_frac=.3, contract_corr=contract_corr_windows['Rolling'], bt_models=[testing_model])

In [ ]:
plot_margin_positions_profit(bt_test, bt_data[testing_model], model=testing_model)

### Hyperparameter grid search

In [ ]:
import itertools

In [ ]:
# Define the parameter ranges
leverages = np.arange(5.0, 15.0, 1) 
kelly_fracs = np.arange(0.1, 1.1, 0.2)
testing_corr_windows = ['Train', '252', 'Rolling', '252_LW', 'Rolling_LW', 'Train_LW']
results = {}
testing_model = 'APARCH'

# Create the grid of all parameter combinations
param_grid = list(itertools.product(leverages, kelly_fracs, testing_corr_windows))
print(f"Starting grid search for {len(param_grid)} parameter combinations...")

for leverage, kelly_frac, contract_corr in tqdm(param_grid):
    # The dictionary key is now a tuple of the parameters
    param_key = (leverage, kelly_frac, contract_corr)
    try:
        bt_test, _ = multi_asset_backtest(max_leverage=leverage, kelly_frac=kelly_frac, bt_models=[testing_model], contract_corr=contract_corr_windows[contract_corr])

        total_value_col = f'total_value_{testing_model}'
        final_value = bt_test[total_value_col].iloc[-1]
        sharpe, mdd = sharpe_and_mdd(bt_test, total_value_col, col_rf="^TNX_close")

        results[param_key] = (final_value, sharpe)
    except Exception as e:
        print(f"Backtest failed for params {param_key}. Reason: {e}")
        results[param_key] = (np.nan, np.nan)

In [ ]:
# Filter out failed runs
valid_results = {k: v for k, v in results.items() if isinstance(v, tuple) and not np.isnan(v[1])}

if valid_results:
    # Find the parameter tuple (key) that corresponds to the maximum Sharpe ratio (the 2nd element of the value)
    best_params = max(valid_results, key=lambda k: valid_results[k][1])
    
    # Unpack the parameters and the results
    best_leverage, best_kelly_frac, best_corr = best_params
    best_total_value, best_sharpe_ratio = valid_results[best_params]

    print("\n--- Optimization Complete (based on Sharpe Ratio) ---")
    print(f"Best Sharpe Ratio: {best_sharpe_ratio:.4f}")
    print(f"Total Value at Best Sharpe: ${best_total_value:,.2f}")
    print(f"Achieved with Max Leverage: {best_leverage:.2f}")
    print(f"Achieved with Kelly Fraction: {best_kelly_frac:.2f}")
    print(f"Achieved with Correlation Window: {best_corr}")

else:
    print("\nAll backtests failed.")

# --- Better Analysis with Pandas ---
print("\n--- Top 10 Results ---")
# Convert results to a DataFrame for easier analysis
records = []
for params, metrics in valid_results.items():
    records.append({
        'Leverage': params[0],
        'KellyFrac': params[1],
        'CorrWindow': params[2],
        'FinalValue': metrics[0],
        'SharpeRatio': metrics[1]
    })
results_df = pd.DataFrame(records)
print(results_df.sort_values(by='SharpeRatio', ascending=False).head(30).round(4))